Preprocessing

In [ ]:
# MAFA Dataset Exploration / Visualization

# Load the .mat file
train_data = scipy.io.loadmat(r'C:\Users\alexw\OneDrive\Documents\03_Education\University_Programming\Python\Big_Data\Coursework\Datasets\MAFA\MAFA-Label-Train\LabelTrainAll.mat')

# Convert the data to a DataFrame
df = pd.DataFrame(train_data['label_train'][0])
df

# Define the path to the images
image_path = r'C:\Users\alexw\OneDrive\Documents\03_Education\University_Programming\Python\Big_Data\Coursework\Datasets\MAFA\MAFA-Label-Train\train-images'

# Function to display image with bounding boxes and eye positions
def display_image_with_features(row):
    # Load the image
    img_name = row['imgName'][0]
    image = plt.imread(image_path + '\\' + img_name)
    
    # Display the image
    plt.imshow(image)
    
    # Get the face bounding box
    face_bbox = row['label'][0][:4]
    x, y, w, h = face_bbox
    
    # Draw face bounding box
    plt.gca().add_patch(patches.Rectangle((x, y), w, h, linewidth=2, edgecolor='r', facecolor='none'))
    
    # Get the bounding box of the occluder
    occluder_bbox = row['label'][0][8:12]
    if not np.all(occluder_bbox == -1):
        ox, oy, ow, oh = occluder_bbox
        # Draw occluder bounding box
        plt.gca().add_patch(patches.Rectangle((x + ox, y + oy), ow, oh, linewidth=2, edgecolor='b', facecolor='none'))
    
    # Get the bounding box of the glasses
    glasses_bbox = row['label'][0][-4:]
    if not np.all(glasses_bbox == -1):
        gx, gy, gw, gh = glasses_bbox
        # Draw glasses bounding box
        plt.gca().add_patch(patches.Rectangle((x + gx, y + gy), gw, gh, linewidth=2, edgecolor='g', facecolor='none'))
    
    # Get the positions of the eyes (relative to the face bounding box)
    eye1_x, eye1_y, eye2_x, eye2_y = row['label'][0][4:8]
    
    # Draw crosses for the positions of the eyes
    plt.plot(eye1_x, eye1_y, 'g+', markersize=15)  # Green cross for eye 1
    plt.plot(eye2_x, eye2_y, 'g+', markersize=15)  # Green cross for eye 2
    
    # Determine whether the person is wearing a mask or not
    occ_type = row['label'][0][12]
    # plt.title('Mask' if occ_type == 1 else f'No Mask - {occ_type}')
    plt.title('Simple Occluder' if occ_type == 1 else 'Complex Occluder' if occ_type == 2 else 'Human Body' if occ_type == 3 else 'Unknown Occluder Type')

    # Additional information below the image
    additional_info = """Face Bounding Box (x, y, w, h): {}\nEye Positions (x1, y1), (x2, y2): ({}, {}), ({}, {})\nOccluder Bounding Box (x, y, w, h): {}\nOcc Type: {}\nOcc Degree: {}\nGender: {}\nRace: {}\nOrientation: {}\nGlasses Bounding Box (x, y, w, h): {}\n""".format(
        (x, y, w, h),
        eye1_x, eye1_y, eye2_x, eye2_y,
        occluder_bbox,
        row['label'][0][11],
        row['label'][0][13],
        row['label'][0][14],
        row['label'][0][15],
        row['label'][0][16],
        row['label'][0][-4:]
    )

    # plt.text(0, 0, additional_info, fontsize=10, ha='left', va='top')

def show_random_faces():
    plt.figure(figsize=(15, 10))
    for i in range(5):
        random_idx = np.random.randint(len(df))  # Randomly select an index
        row = df.iloc[random_idx].copy()  # Get a copy of the row
        plt.subplot(5, 5, i+1)
        display_image_with_features(row)
        plt.axis('off')

    plt.show()
    
def show_selected_faces(rows):
    plt.figure(figsize=(15, 10))
    for i, idx in enumerate(rows, start=1):
        row = df.iloc[idx].copy()
        plt.subplot(1, 5, i)
        display_image_with_features(row)
        plt.axis('off')
        
    plt.show()
    
rows = [0, 1, 2, 3, 4]
show_selected_faces(rows)



In [ ]:
# Resizes images and adjusts labels accordingly. Uses a size of 224 x 224.

import pandas as pd
import numpy as np
import scipy.io
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2

# Load the .mat file
train_data = scipy.io.loadmat(r'C:\Users\alexw\OneDrive\Documents\03_Education\University_Programming\Python\Big_Data\Coursework\Datasets\MAFA\MAFA-Label-Train\LabelTrainAll.mat')

# Convert the data to a DataFrame
df = pd.DataFrame(train_data['label_train'][0])

# Define the path to the images
image_path = r'C:\Users\alexw\OneDrive\Documents\03_Education\University_Programming\Python\Big_Data\Coursework\Datasets\MAFA\MAFA-Label-Train\train-images'

# Function to resize image and update labels
def resize_image_and_labels(row, target_size):
    # Load the image
    img_name = row['imgName'][0]
    img = cv2.imread(image_path + '\\' + img_name)
    
    # Resize image
    resized_img = cv2.resize(img, target_size)
    
    # Calculate scaling factors for resizing
    fx = target_size[0] / img.shape[1]
    fy = target_size[1] / img.shape[0]
    
    # Update face and occluder coordinates
    row['label'][0][:4] = np.round(row['label'][0][:4] * np.array([fx, fy, fx, fy])).astype(int)
    row['label'][0][8:12] = np.round(row['label'][0][8:12] * np.array([fx, fy, fx, fy])).astype(int)
    row['label'][0][-4:] = np.round(row['label'][0][-4:] * np.array([fx, fy, fx, fy])).astype(int)
    
    # Update eye coordinates
    row['label'][0][4:6] = np.round(row['label'][0][4:6] * np.array([fx, fy])).astype(int)
    row['label'][0][6:8] = np.round(row['label'][0][6:8] * np.array([fx, fy])).astype(int)

    return resized_img, row

# Function to display image with bounding boxes and eye positions
def display_image_with_features(img, row):
    # Display the image
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    
    # Get the face bounding box
    face_bbox = row['label'][0][:4]
    x, y, w, h = face_bbox
    
    # Draw face bounding box
    plt.gca().add_patch(patches.Rectangle((x, y), w, h, linewidth=2, edgecolor='r', facecolor='none'))
    
    # Get the bounding box of the occluder
    occluder_bbox = row['label'][0][8:12]
    if not np.all(occluder_bbox == -1):
        ox, oy, ow, oh = occluder_bbox
        # Draw occluder bounding box
        plt.gca().add_patch(patches.Rectangle((x + ox, y + oy), ow, oh, linewidth=2, edgecolor='b', facecolor='none'))
    
    # Get the bounding box of the glasses
    glasses_bbox = row['label'][0][-4:]
    if not np.all(glasses_bbox == -1):
        gx, gy, gw, gh = glasses_bbox
        # Draw glasses bounding box
        plt.gca().add_patch(patches.Rectangle((x + gx, y + gy), gw, gh, linewidth=2, edgecolor='g', facecolor='none'))
    
    # Get the positions of the eyes (relative to the face bounding box)
    eye1_x, eye1_y, eye2_x, eye2_y = row['label'][0][4:8]
    
    # Draw crosses for the positions of the eyes
    plt.plot(eye1_x, eye1_y, 'g+', markersize=15)  # Green cross for eye 1
    plt.plot(eye2_x, eye2_y, 'g+', markersize=15)  # Green cross for eye 2
    
    # Determine whether the person is wearing a mask or not
    occ_type = row['label'][0][12]
    plt.title('Simple Occluder' if occ_type == 1 else 'Complex Occluder' if occ_type == 2 else 'Human Body' if occ_type == 3 else 'Unknown Occluder Type')

# Display random images with bounding boxes and eye positions
plt.figure(figsize=(15, 10))
for i in range(5):
    random_idx = np.random.randint(len(df))  # Randomly select an index
    row = df.iloc[random_idx].copy()  # Get a copy of the row
    resized_img, row = resize_image_and_labels(row, (224, 224))
    plt.subplot(1, 5, i+1)
    display_image_with_features(resized_img, row)
    plt.axis('off')

plt.show()


In [ ]:
# Resizes images and adjusts labels accordingly v2...uses csv data instead of .mat data.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2

# Load the CSV file
csv_path = 'csv/MAFA_Training_Data_Unstructured.csv'
df = pd.read_csv(csv_path)
image_path = r'C:\Users\alexw\OneDrive\Documents\03_Education\University_Programming\Python\Big_Data\Coursework\Datasets\MAFA\MAFA-Label-Train\train-images'

# Function to resize image and update labels
def resize_image_and_labels(row, target_size, output_dir=None):
    # Load the image
    img_name = row['imgName']
    img = cv2.imread(image_path + '\\' + img_name)
    
    # Resize image
    resized_img = cv2.resize(img, target_size)
    
    # Calculate scaling factors for resizing
    fx = target_size[0] / img.shape[1]
    fy = target_size[1] / img.shape[0]
    
    # Update face and occluder coordinates
    row['face_x'] = np.round(row['face_x'] * fx).astype(int)
    row['face_y'] = np.round(row['face_y'] * fy).astype(int)
    row['face_w'] = np.round(row['face_w'] * fx).astype(int)
    row['face_h'] = np.round(row['face_h'] * fy).astype(int)
    
    row['eye1_x'] = np.round(row['eye1_x'] * fx).astype(int)
    row['eye1_y'] = np.round(row['eye1_y'] * fy).astype(int)
    row['eye2_x'] = np.round(row['eye2_x'] * fx).astype(int)
    row['eye2_y'] = np.round(row['eye2_y'] * fy).astype(int)
    
    row['occluder_x'] = np.round(row['occluder_x'] * fx).astype(int)
    row['occluder_y'] = np.round(row['occluder_y'] * fy).astype(int)
    row['occluder_w'] = np.round(row['occluder_w'] * fx).astype(int)
    row['occluder_h'] = np.round(row['occluder_h'] * fy).astype(int)

    return resized_img, row

# Function to display image with bounding boxes and eye positions
def display_resized_image_with_features(img, row):
    # Display the image
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    
    # Draw face bounding box
    plt.gca().add_patch(patches.Rectangle((row['face_x'], row['face_y']), row['face_w'], row['face_h'], linewidth=2, edgecolor='r', facecolor='none'))
    
    # Draw occluder bounding box
    plt.gca().add_patch(patches.Rectangle((row['face_x'] + row['occluder_x'], row['face_y'] + row['occluder_y']), row['occluder_w'], row['occluder_h'], linewidth=2, edgecolor='b', facecolor='none'))
    
    # Draw glasses bounding box
    plt.gca().add_patch(patches.Rectangle((row['face_x'] + row['glasses_x'], row['face_y'] + row['glasses_y']), row['glasses_w'], row['glasses_h'], linewidth=2, edgecolor='g', facecolor='none'))
    
    # Draw crosses for the positions of the eyes
    plt.plot(row['eye1_x'], row['eye1_y'], 'g+', markersize=15)  # Green cross for eye 1
    plt.plot(row['eye2_x'], row['eye2_y'], 'g+', markersize=15)  # Green cross for eye 2
    
    # Determine whether the person is wearing a mask or not
    occ_type = row['occluder_type']
    plt.title('Simple Occluder' if occ_type == 1 else 'Complex Occluder' if occ_type == 2 else 'Human Body' if occ_type == 3 else 'Unknown Occluder Type')

# Display random images with bounding boxes and eye positions
def resize_random_faces():
    plt.figure(figsize=(15, 10))
    for i in range(5):
        random_idx = np.random.randint(len(df))  # Randomly select an index
        row = df.iloc[random_idx].copy()  # Get a copy of the row
        resized_img, row = resize_image_and_labels(row, (224, 224))
        plt.subplot(5, 5, i+1)
        display_resized_image_with_features(resized_img, row)
        plt.axis('off')

    plt.show()
    
def resize_selected_faces(rows):
    plt.figure(figsize=(15, 10))
    for i, idx in enumerate(rows, start=1):
        row = df.iloc[idx].copy()
        resized_img, row = resize_image_and_labels(row, (224, 224))
        plt.subplot(1, 5, i)
        display_resized_image_with_features(resized_img, row)
        plt.axis('off')
        
    plt.show()

# random_idx = np.random.randint(len(df))  # Randomly select an index
# rows =- df.iloc[random_idx].copy()
rows = [0, 1, 2, 3, 18]
resize_random_faces()
# resize_selected_faces(rows)
# This currently moves the location of the bbxs each times it runs and i'm not entirely sure why. Also the face of 4 is being shifted up for some reason.

In [ ]:
# Adjusts the labels of all images accordingly so 224, 224 labels and stores in a csv

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2

# Load the CSV file
csv_path = 'csv/MAFA_Training_Data_Unstructured.csv'
df = pd.read_csv(csv_path)
# image_path = r'C:\Users\alexw\OneDrive\Documents\03_Education\University_Programming\Python\Big_Data\Coursework\Datasets\MAFA\MAFA-Label-Train\train-images'

# Function to resize image and update labels
def resize_image_and_labels(row, target_size, output_dir=None):
    # Load the image
    img_name = row['imgName']
    img = cv2.imread(image_path + '\\' + img_name)
    
    # Resize image
    resized_img = cv2.resize(img, target_size)
    
    # Calculate scaling factors for resizing
    fx = target_size[0] / img.shape[1]
    fy = target_size[1] / img.shape[0]
    
    # Update face and occluder coordinates
    row['face_x'] = np.round(row['face_x'] * fx).astype(int)
    row['face_y'] = np.round(row['face_y'] * fy).astype(int)
    row['face_w'] = np.round(row['face_w'] * fx).astype(int)
    row['face_h'] = np.round(row['face_h'] * fy).astype(int)
    
    row['eye1_x'] = np.round(row['eye1_x'] * fx).astype(int)
    row['eye1_y'] = np.round(row['eye1_y'] * fy).astype(int)
    row['eye2_x'] = np.round(row['eye2_x'] * fx).astype(int)
    row['eye2_y'] = np.round(row['eye2_y'] * fy).astype(int)
    
    row['occluder_x'] = np.round(row['occluder_x'] * fx).astype(int)
    row['occluder_y'] = np.round(row['occluder_y'] * fy).astype(int)
    row['occluder_w'] = np.round(row['occluder_w'] * fx).astype(int)
    row['occluder_h'] = np.round(row['occluder_h'] * fy).astype(int)

    return resized_img, row

def resize_all_image_labels():
    # Resize all images and update labels
    resized_rows = []
    for _, row in df.iterrows():
        resized_img, resized_row = resize_image_and_labels(row, (224, 224))
        resized_rows.append(resized_row)

    # Create DataFrame from resized rows
    resized_df = pd.DataFrame(resized_rows)

    # Save resized DataFrame to CSV
    resized_csv_path = 'csv/MAFA_Training_Data_Unstructured_Resized.csv'
    resized_df.to_csv(resized_csv_path, index=False)
    
    print("Image Labels have all been resized and saved to csv/MAFA_Training_Data_Unstructured_Resized.csv")
    
# resize_all_image_labels()


In [ ]:
# Resizes images and adjusts labels accordingly v2...

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2

# Load the CSV file
csv_path = 'csv/MAFA_Training_Data_Unstructured.csv'
df = pd.read_csv(csv_path)
image_path = r'C:\Users\alexw\OneDrive\Documents\03_Education\University_Programming\Python\Big_Data\Coursework\Datasets\MAFA\MAFA-Label-Train\train-images'

# Function to resize image and update labels
def resize_image_and_labels(row, target_size, output_dir=None):
    # Load the image
    img_name = row['imgName']
    img = cv2.imread(image_path + '\\' + img_name)
    
    # Resize image
    resized_img = cv2.resize(img, target_size)
    
    # Calculate scaling factors for resizing
    fx = target_size[0] / img.shape[1]
    fy = target_size[1] / img.shape[0]
    
    # Update face and occluder coordinates
    row['face_x'] = np.round(row['face_x'] * fx).astype(int)
    row['face_y'] = np.round(row['face_y'] * fy).astype(int)
    row['face_w'] = np.round(row['face_w'] * fx).astype(int)
    row['face_h'] = np.round(row['face_h'] * fy).astype(int)
    
    row['eye1_x'] = np.round(row['eye1_x'] * fx).astype(int)
    row['eye1_y'] = np.round(row['eye1_y'] * fy).astype(int)
    row['eye2_x'] = np.round(row['eye2_x'] * fx).astype(int)
    row['eye2_y'] = np.round(row['eye2_y'] * fy).astype(int)
    
    row['occluder_x'] = np.round(row['occluder_x'] * fx).astype(int)
    row['occluder_y'] = np.round(row['occluder_y'] * fy).astype(int)
    row['occluder_w'] = np.round(row['occluder_w'] * fx).astype(int)
    row['occluder_h'] = np.round(row['occluder_h'] * fy).astype(int)

    return resized_img, row

# Function to display image with bounding boxes and eye positions
def display_resized_image_with_features(img, row):
    # Display the image
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    
    # Draw face bounding box
    plt.gca().add_patch(patches.Rectangle((row['face_x'], row['face_y']), row['face_w'], row['face_h'], linewidth=2, edgecolor='r', facecolor='none'))
    
    # Draw occluder bounding box
    plt.gca().add_patch(patches.Rectangle((row['face_x'] + row['occluder_x'], row['face_y'] + row['occluder_y']), row['occluder_w'], row['occluder_h'], linewidth=2, edgecolor='b', facecolor='none'))
    
    # Draw glasses bounding box
    plt.gca().add_patch(patches.Rectangle((row['face_x'] + row['glasses_x'], row['face_y'] + row['glasses_y']), row['glasses_w'], row['glasses_h'], linewidth=2, edgecolor='g', facecolor='none'))
    
    # Draw crosses for the positions of the eyes
    plt.plot(row['eye1_x'], row['eye1_y'], 'g+', markersize=15)  # Green cross for eye 1
    plt.plot(row['eye2_x'], row['eye2_y'], 'g+', markersize=15)  # Green cross for eye 2
    
    # Determine whether the person is wearing a mask or not
    occ_type = row['occluder_type']
    plt.title('Simple Occluder' if occ_type == 1 else 'Complex Occluder' if occ_type == 2 else 'Human Body' if occ_type == 3 else 'Unknown Occluder Type')

# Display random images with bounding boxes and eye positions
def resize_random_faces():
    plt.figure(figsize=(15, 10))
    for i in range(5):
        random_idx = np.random.randint(len(df))  # Randomly select an index
        row = df.iloc[random_idx].copy()  # Get a copy of the row
        resized_img, row = resize_image_and_labels(row, (224, 224))
        plt.subplot(5, 5, i+1)
        display_resized_image_with_features(resized_img, row)
        plt.axis('off')

    plt.show()
    
def resize_selected_faces(rows):
    plt.figure(figsize=(15, 10))
    for i, idx in enumerate(rows, start=1):
        row = df.iloc[idx].copy()
        resized_img, row = resize_image_and_labels(row, (224, 224))
        plt.subplot(1, 5, i)
        display_resized_image_with_features(resized_img, row)
        plt.axis('off')
        
    plt.show()


In [ ]:
# Puts the data (unstructured) into a csv in order to apply dimensionality reduction on it.

import pandas as pd
import numpy as np
import scipy.io

# Load the .mat file
train_data = scipy.io.loadmat(r'C:\Users\alexw\OneDrive\Documents\03_Education\University_Programming\Python\Big_Data\Coursework\Datasets\MAFA\MAFA-Label-Train\LabelTrainAll.mat')
image_dir = r'C:\Users\alexw\OneDrive\Documents\03_Education\University_Programming\Python\Big_Data\Coursework\Datasets\MAFA\MAFA-Label-Train\train-images'

# Convert the data to a DataFrame
df = pd.DataFrame(train_data['label_train'][0])

# Preprocess DataFrame to remove square brackets
df['orgImgName'] = df['orgImgName'].apply(lambda x: x[0])
df['imgName'] = df['imgName'].apply(lambda x: x[0])
df['label'] = df['label'].apply(lambda x: ', '.join(map(str, x[0])))

# Structure the data
label_columns = ['face_x', 'face_y', 'face_w', 'face_h', 'eye1_x', 'eye1_y', 'eye2_x', 'eye2_y', 
                 'occluder_x', 'occluder_y', 'occluder_w', 'occluder_h', 'occ_type', 'occ_degree', 
                 'gender', 'race', 'orientation', 'glasses_x', 'glasses_y', 'glasses_w', 'glasses_h']
df[label_columns] = df['label'].str.split(', ', expand=True)

# Convert the bounding box coordinates from strings to floats
for col in label_columns:
    df[col] = df[col].astype(float)

df.drop(columns=['label'], inplace=True)

# Define columns for the CSV
csv_columns = ['orgImgName', 'imgName', 'face_x', 'face_y', 'face_w', 'face_h', 
               'eye1_x', 'eye1_y', 'eye2_x', 'eye2_y', 'occluder_x', 'occluder_y', 
               'gender', 'race', 'orientation', 'glasses_x', 'glasses_y', 'glasses_w', 'glasses_h']

# Save the DataFrame to a CSV file
df[csv_columns].to_csv('csv/MAFA_training_data_unstructured.csv', index=False)

print("CSV file has been successfully generated.")

In [ ]:
# Correctly structures both training data and the test data into their own csvs

import scipy.io
import pandas as pd

# Load the .mat files
test_data = scipy.io.loadmat(r'C:\Users\alexw\OneDrive\Documents\03_Education\University_Programming\Python\Big_Data\Coursework\Datasets\MAFA\MAFA-Label-Test\LabelTestAll.mat')
train_data = scipy.io.loadmat(r'C:\Users\alexw\OneDrive\Documents\03_Education\University_Programming\Python\Big_Data\Coursework\Datasets\MAFA\MAFA-Label-Train\LabelTrainAll.mat')

# Convert the test data to a DataFrame
test_label_data = test_data['LabelTest'][0]
test_rows = []
for img_name, label_array in test_label_data:
    img_name = img_name[0]
    for label in label_array:
        row = [img_name] + label.tolist()
        test_rows.append(row)

# Create a DataFrame for test data
test_df = pd.DataFrame(test_rows, columns=['imgName', 'face_x', 'face_y', 'face_w', 'face_h', 'face_type', 'occluder_x', 'occluder_y', 'occluder_w', 'occluder_h', 'occluder_type', 'occluder_degree', 'gender', 'race', 'orientation', 'glasses_x', 'glasses_y', 'glasses_w', 'glasses_h'])

# Convert the bounding box coordinates from strings to floats for test data
for col in test_df.columns[1:]:
    test_df[col] = test_df[col].astype(float)

# Save the test DataFrame to a CSV file
test_df.to_csv('csv/MAFA_test_data.csv', index=False)
print("Test CSV file has been successfully generated.")

# Convert the training data to a DataFrame
train_label_data = train_data['label_train'][0]
train_rows = []
for item in train_label_data:
    org_img_name = item[0]
    img_name = item[1]
    label_array = item[2]
    for label in label_array:
        row = [img_name[0]] + label.tolist()
        train_rows.append(row)

# Create a DataFrame for training data
train_df = pd.DataFrame(train_rows, columns=['imgName', 'face_x', 'face_y', 'face_w', 'face_h', 'eye1_x', 'eye1_y', 'eye2_x', 'eye2_y', 'occluder_x', 'occluder_y', 'occluder_w', 'occluder_h', 'occluder_type', 'occluder_degree', 'gender', 'race', 'orientation', 'glasses_x', 'glasses_y', 'glasses_w', 'glasses_h'])

# Convert the bounding box coordinates from strings to floats for training data
for col in train_df.columns[1:]:
    train_df[col] = train_df[col].astype(float)

# Save the training DataFrame to a CSV file
train_df.to_csv('csv/MAFA_training_data.csv', index=False)
print("Training CSV file has been successfully generated.")

In [ ]:
# Correctly structures the test data into a csv

import scipy.io
import pandas as pd
import numpy as np

# Load the .mat file
test_data = scipy.io.loadmat(r'C:\Users\alexw\OneDrive\Documents\03_Education\University_Programming\Python\Big_Data\Coursework\Datasets\MAFA\MAFA-Label-Test\LabelTestAll.mat')
image_dir = r'C:\Users\alexw\OneDrive\Documents\03_Education\University_Programming\Python\Big_Data\Coursework\Datasets\MAFA\MAFA-Label-Train\test-images'

# Convert the data to a DataFrame
label_data = test_data['LabelTest'][0]
rows = []
for img_name, label_array in label_data:
    img_name = img_name[0]  # Get rid of the extra square brackets
    for label in label_array:
        row = [img_name] + label.tolist()
        rows.append(row)

# Create a DataFrame from the list of rows
df = pd.DataFrame(rows, columns=['imgName', 'face_x', 'face_y', 'face_w', 'face_h', 'face_type', 'occluder_x', 'occluder_y', 'occluder_w', 'occluder_h', 'occluder_type', 'occluder_degree', 'gender', 'race', 'orientation', 'glasses_x', 'glasses_y', 'glasses_w', 'glasses_h'])

# Convert the bounding box coordinates from strings to floats
for col in df.columns[1:]:
    df[col] = df[col].astype(float)

# Save the DataFrame to a CSV file
df.to_csv('csv/MAFA_test_data.csv', index=False)
print("CSV file has been successfully generated.")

Preprocessing - Dimensionality Analysis, Reduction, and Visualization

In [ ]:
# Used - Best Scatterplot representation of the PCA distribution

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load the data from CSV
df = pd.read_csv("csv/MAFA_Training_data_PCA.csv")

# Limit the number of unique image names for the legend
max_legend_entries = 20
unique_img_names = df['orgImgName'].unique()[:max_legend_entries]

# sampled_df = df.sample(n=2000)

# Create a scatter plot with principal components
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x='PC1', y='PC2', hue='orgImgName', palette='viridis')
plt.title('PCA Visualization of MAFA Training Data')
plt.xlabel('Principal Component 1') # Principal Component 1 is the Face BBX
plt.ylabel('Principal Component 2') # Principal Component 1 is the Occluder BBX
plt.legend(title='Image Name', labels=unique_img_names, loc='upper right')
plt.grid(True)
plt.show()


In [ ]:
# Unused - Best Scatterplot representation of the PCA distribution - Just the Scatterplot

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load the data from CSV
df = pd.read_csv("csv/MAFA_Training_data_PCA.csv")

# Limit the number of unique image names for the legend
max_legend_entries = 20
unique_img_names = df['orgImgName'].unique()[:max_legend_entries]

# sampled_df = df.sample(n=2000)

# Create a scatter plot with principal components
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x='PC1', y='PC2', hue='orgImgName', palette='viridis')
plt.title('PCA Visualization of MAFA Training Data')
plt.xlabel('Principal Component 1') # Principal Component 1 is the Face BBX
plt.ylabel('Principal Component 2') # Principal Component 1 is the Occluder BBX
plt.legend(title='Image Name', labels=unique_img_names, loc='upper right')
plt.grid(True)
plt.show()


In [ ]:
# Used - Plotting of the PCA results and using K-means mappings...

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

# Load the transformed data from the CSV file
df = pd.read_csv('csv/MAFA_Training_data_PCA.csv')

# Limit the number of unique image names for the legend
max_legend_entries = 20
unique_img_names = df['orgImgName'].unique()[:max_legend_entries]

# sampled_df = df.sample(n=2000) # Can be used to select a random sample of size n

# Create a scatter plot with principal components
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x='PC1', y='PC2', hue='orgImgName', palette='viridis')
plt.title('PCA Visualization of MAFA Training Data')
plt.xlabel('Principal Component 1') # Principal Component 1 is the Face BBX
plt.ylabel('Principal Component 2') # Principal Component 1 is the Occluder BBX
plt.legend(title='Image Name', labels=unique_img_names, loc='upper right')
plt.grid(True)
plt.show()

# Assuming df_pca contains the data with PC1 and PC2 columns
# Perform K-means clustering
kmeans = KMeans(n_clusters=3)  # Specify the number of clusters
clusters = kmeans.fit_predict(df[['PC1', 'PC2']])

# Visualize clusters
plt.figure(figsize=(8, 6))
plt.scatter(df['PC1'], df['PC2'], c=clusters, cmap='viridis')
plt.title('K-means Clustering of PCA Data')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.colorbar(label='Cluster')
plt.show()



In [ ]:
# Resizes images and adjusts labels accordingly. Uses a size of 224 x 224.

import pandas as pd
import numpy as np
import scipy.io
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2

# Function to resize image and update labels
def resize_image_and_labels(row, target_size):
    # Load the image
    img_name = row['imgName'][0]
    img = cv2.imread(image_path + '\\' + img_name)
    
    # Resize image
    resized_img = cv2.resize(img, target_size)
    
    # Calculate scaling factors for resizing
    fx = target_size[0] / img.shape[1]
    fy = target_size[1] / img.shape[0]
    
    # Update face and occluder coordinates
    row['label'][0][:4] = np.round(row['label'][0][:4] * np.array([fx, fy, fx, fy])).astype(int)
    row['label'][0][8:12] = np.round(row['label'][0][8:12] * np.array([fx, fy, fx, fy])).astype(int)
    row['label'][0][-4:] = np.round(row['label'][0][-4:] * np.array([fx, fy, fx, fy])).astype(int)
    
    # Update eye coordinates
    row['label'][0][4:6] = np.round(row['label'][0][4:6] * np.array([fx, fy])).astype(int)
    row['label'][0][6:8] = np.round(row['label'][0][6:8] * np.array([fx, fy])).astype(int)

    return resized_img, row

# Function to display image with bounding boxes and eye positions
def display_image_with_features(img, row):
    # Display the image
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    
    # Get the face bounding box
    face_bbox = row['label'][0][:4]
    x, y, w, h = face_bbox
    
    # Draw face bounding box
    plt.gca().add_patch(patches.Rectangle((x, y), w, h, linewidth=2, edgecolor='r', facecolor='none'))
    
    # Get the bounding box of the occluder
    occluder_bbox = row['label'][0][8:12]
    if not np.all(occluder_bbox == -1):
        ox, oy, ow, oh = occluder_bbox
        # Draw occluder bounding box
        plt.gca().add_patch(patches.Rectangle((x + ox, y + oy), ow, oh, linewidth=2, edgecolor='b', facecolor='none'))
    
    # Get the bounding box of the glasses
    glasses_bbox = row['label'][0][-4:]
    if not np.all(glasses_bbox == -1):
        gx, gy, gw, gh = glasses_bbox
        # Draw glasses bounding box
        plt.gca().add_patch(patches.Rectangle((x + gx, y + gy), gw, gh, linewidth=2, edgecolor='g', facecolor='none'))
    
    # Get the positions of the eyes (relative to the face bounding box)
    eye1_x, eye1_y, eye2_x, eye2_y = row['label'][0][4:8]
    
    # Draw crosses for the positions of the eyes
    plt.plot(eye1_x, eye1_y, 'g+', markersize=15)  # Green cross for eye 1
    plt.plot(eye2_x, eye2_y, 'g+', markersize=15)  # Green cross for eye 2
    
    # Determine whether the person is wearing a mask or not
    occ_type = row['label'][0][12]
    plt.title('Simple Occluder' if occ_type == 1 else 'Complex Occluder' if occ_type == 2 else 'Human Body' if occ_type == 3 else 'Unknown Occluder Type')

# Display random images with bounding boxes and eye positions
plt.figure(figsize=(15, 10))
for i in range(5):
    random_idx = np.random.randint(len(df))  # Randomly select an index
    row = df.iloc[random_idx].copy()  # Get a copy of the row
    resized_img, row = resize_image_and_labels(row, (224, 224))
    plt.subplot(1, 5, i+1)
    display_image_with_features(resized_img, row)
    plt.axis('off')

plt.show()


Preprocessing - Structuring of the data

In [ ]:
# Data Structuring and Preprocessing

import pandas as pd
import numpy as np
import scipy.io

# Load the .mat file
train_data = scipy.io.loadmat(r'C:\Users\alexw\OneDrive\Documents\03_Education\University_Programming\Python\Big_Data\Coursework\Datasets\MAFA\MAFA-Label-Train\LabelTrainAll.mat')

# Convert the data to a DataFrame
df = pd.DataFrame(train_data['label_train'][0])

# Preprocess DataFrame to remove square brackets
df['orgImgName'] = df['orgImgName'].apply(lambda x: x[0])
df['imgName'] = df['imgName'].apply(lambda x: x[0])
df['label'] = df['label'].apply(lambda x: ', '.join(map(str, x[0])))

### Missing Value validation

# Identify missing values
missing_values = df.isnull().sum()
print("Missing Values:")
print(missing_values)

# Fill in the missing values with the mean value.
# df.fillna(df.mean(), inplace=True) # Cannot be done with images, thus we just remove the row completely.
df.dropna(inplace=True)

### Validating the values column to ensure correct number of elements

# Validate the number of elements in the 'label' column
valid_label_count = df['label'].apply(lambda x: len(x.split(', ')) == 21)
missing_values = valid_label_count.value_counts()[True]  # Count missing values
print("\nNumber of elements with expected label array size:", missing_values)
# print("Missing values:", missing_values)

### Double checking for duplicate values

# Check for duplicate rows and removes them if they exist
duplicates = df[df.duplicated()]
print(f"Duplicates duplicated: ${duplicates}")
df.drop_duplicates(inplace=True) # Removes the duplicates

### Removing outliers

# Identify outliers using Z-score...The Z-score measures how many deviations within the mean a data point is. 
# This doesn't work with MaskedFace recognition because all of the data is unique to that particular image. There is no way of guessing the location of the face.
# z_scores = (df['column_name'] - df['column_name'].mean()) / df['column_name'].std()
# outliers = df[(z_scores > 3) | (z_scores < -3)]

# df = df[(z_scores < 3) & (z_scores > -3)]

# Save the DataFrame to a CSV file without square brackets
df.to_csv('csv/MAFA_Label_Train.csv', index=False)

# Display the cleaned DataFrame
print("\nCleaned DataFrame:")
print(df.head())


In [ ]:
# Data Structuring and Preprocessing v2

import pandas as pd
import numpy as np
import scipy.io

# Load the .mat file
train_data = scipy.io.loadmat(r'C:\Users\alexw\OneDrive\Documents\03_Education\University_Programming\Python\Big_Data\Coursework\Datasets\MAFA\MAFA-Label-Train\LabelTrainAll.mat')

# Convert the data to a DataFrame
df = pd.DataFrame(train_data['label_train'][0])

# Preprocess DataFrame to remove square brackets
df['orgImgName'] = df['orgImgName'].apply(lambda x: x[0])
df['imgName'] = df['imgName'].apply(lambda x: x[0])
df['label'] = df['label'].apply(lambda x: ', '.join(map(str, x[0])))

### Missing Value validation

# Identify missing values
missing_values = df.isnull().sum()
print("Missing Values:")
print(missing_values)

# Fill in the missing values with the mean value.
# df.fillna(df.mean(), inplace=True) # Cannot be done with images, thus we just remove the row completely.
df.dropna(inplace=True)

### Validating the values column to ensure correct number of elements

# Validate the number of elements in the 'label' column
valid_label_count = df['label'].apply(lambda x: len(x.split(', ')) == 21)
missing_values = valid_label_count.value_counts()[True]  # Count missing values
print("\nNumber of elements with expected label array size:", missing_values)
# print("Missing values:", missing_values)

### Double checking for duplicate values

# Check for duplicate rows and removes them if they exist
duplicates = df[df.duplicated()]
print(f"Duplicates duplicated: ${duplicates}")
df.drop_duplicates(inplace=True) # Removes the duplicates

### Structure the data and convert into a csv

# Extract bounding box information
label_columns = ['face_x', 'face_y', 'face_w', 'face_h', 'eye1_x', 'eye1_y', 'eye2_x', 'eye2_y', 'occluder_x', 'occluder_y', 'occluder_w', 'occluder_h', 'occ_type', 'occ_degree', 'gender', 'race', 'orientation', 'glasses_x', 'glasses_y', 'glasses_w', 'glasses_h']
df[label_columns] = df['label'].str.split(', ', expand=True)

# Create separate columns for bounding boxes
df['face_bbx'] = df[['face_x', 'face_y', 'face_w', 'face_h']].values.tolist()
df['left_eye_bbx'] = df[['eye1_x', 'eye1_y']].values.tolist()
df['right_eye_bbx'] = df[['eye2_x', 'eye2_y']].values.tolist()
df['occluder_bbx'] = df[['occluder_x', 'occluder_y', 'occluder_w', 'occluder_h']].values.tolist()
df['occluder_type'] = df['occ_type']
df['occluder_degree'] = df['occ_degree']
df['face_gender'] = df['gender']
df['face_race'] = df['race']
df['face_orientation'] = df['orientation']
df['glasses_bbx'] = df[['glasses_x', 'glasses_y', 'glasses_w', 'glasses_h']].values.tolist()

# Drop the original columns
df.drop(columns=label_columns, inplace=True)

# Convert the extracted columns to numeric type
df[['face_bbx', 'left_eye_bbx', 'right_eye_bbx', 'occluder_bbx', 'occluder_type', 'occluder_degree', 'face_gender', 'face_race', 'face_orientation', 'glasses_bbx']] = df[['face_bbx', 'left_eye_bbx', 'right_eye_bbx', 'occluder_bbx', 'occluder_type', 'occluder_degree', 'face_gender', 'face_race', 'face_orientation', 'glasses_bbx']].map(np.array)

# Drop the original 'label' column
df.drop(columns=['label'], inplace=True)

# Save the DataFrame to a CSV file without square brackets
df.to_csv('csv/MAFA_Label_Train.csv', index=False)

# Display the cleaned DataFrame
print("\nCleaned and structured DataFrame:")
print(df.head())

In [ ]:
# Data Structuring and Preprocessing v3

import pandas as pd
import numpy as np
import scipy.io
from PIL import Image
import imagehash
import os

# Load the .mat file
train_data = scipy.io.loadmat(r'C:\Users\alexw\OneDrive\Documents\03_Education\University_Programming\Python\Big_Data\Coursework\Datasets\MAFA\MAFA-Label-Train\LabelTrainAll.mat')
image_dir = r'C:\Users\alexw\OneDrive\Documents\03_Education\University_Programming\Python\Big_Data\Coursework\Datasets\MAFA\MAFA-Label-Train\train-images'

# Convert the data to a DataFrame
df = pd.DataFrame(train_data['label_train'][0])

# Preprocess DataFrame to remove square brackets
df['orgImgName'] = df['orgImgName'].apply(lambda x: x[0])
df['imgName'] = df['imgName'].apply(lambda x: x[0])
df['label'] = df['label'].apply(lambda x: ', '.join(map(str, x[0])))

### Missing Value validation

# Identify missing values
missing_values = df.isnull().sum()
print("Missing Values:")
print(missing_values)

# Fill in the missing values with the mean value.
# df.fillna(df.mean(), inplace=True) # Cannot be done with images, thus we just remove the row completely.
df.dropna(inplace=True)

### Validating the values column to ensure correct number of elements

# Validate the number of elements in the 'label' column
valid_label_count = df['label'].apply(lambda x: len(x.split(', ')) == 21)
missing_values = valid_label_count.value_counts()[True]  # Count missing values
print("\nNumber of elements with expected label array size:", missing_values)
# print("Missing values:", missing_values)

### Double checking for duplicate values - Does nothing with them, just a note at the moment.

def find_duplicates_images(image_dir):
    """
    Find duplicate images in a directory based on their perceptual hashes and print them.
    """
    # Dictionary to store hashes and corresponding filenames
    hash_dict = {}
    
    # List to store duplicate image filenames
    duplicate_images = []
    
    # Iterate through all files in the directory
    for filename in os.listdir(image_dir):
        # Check if the file is an image
        if filename.endswith((".jpg", ".png")):
            # Get the full file path
            filepath = os.path.join(image_dir, filename)
            
            # Open the image and calculate its hash
            with Image.open(filepath) as img:
                hash_val = str(imagehash.average_hash(img))
            
            # Check if the hash already exists in the dictionary
            if hash_val in hash_dict:
                # Add the current filename and the filename already stored in hash_dict
                duplicate_images.append((filename, hash_dict[hash_val]))
            else:
                # Add the hash and filename to the dictionary
                hash_dict[hash_val] = filename
                
    # Print the duplicate images
    if duplicate_images:
        print("Duplicate images found:")
        for img1, img2 in duplicate_images:
            print(f"{img2} is a duplicate of {img1}")
    else:
        print("No duplicate images found.")

# Call the function to find and print duplicate images
find_duplicates_images(image_dir)

### Structure the data and convert into a csv

# Extract bounding box information
label_columns = ['face_x', 'face_y', 'face_w', 'face_h', 'eye1_x', 'eye1_y', 'eye2_x', 'eye2_y', 'occluder_x', 'occluder_y', 'occluder_w', 'occluder_h', 'occ_type', 'occ_degree', 'gender', 'race', 'orientation', 'glasses_x', 'glasses_y', 'glasses_w', 'glasses_h']
df[label_columns] = df['label'].str.split(', ', expand=True)

# Create separate columns for bounding boxes
df['face_bbx'] = df[['face_x', 'face_y', 'face_w', 'face_h']].values.tolist()
df['left_eye_bbx'] = df[['eye1_x', 'eye1_y']].values.tolist()
df['right_eye_bbx'] = df[['eye2_x', 'eye2_y']].values.tolist()
df['occluder_bbx'] = df[['occluder_x', 'occluder_y', 'occluder_w', 'occluder_h']].values.tolist()
df['occluder_type'] = df['occ_type']
df['occluder_degree'] = df['occ_degree']
df['face_gender'] = df['gender']
df['face_race'] = df['race']
df['face_orientation'] = df['orientation']
df['glasses_bbx'] = df[['glasses_x', 'glasses_y', 'glasses_w', 'glasses_h']].values.tolist()

# Drop the original columns
df.drop(columns=label_columns, inplace=True)

# Convert the extracted columns to numeric type
df[['face_bbx', 'left_eye_bbx', 'right_eye_bbx', 'occluder_bbx', 'occluder_type', 'occluder_degree', 'face_gender', 'face_race', 'face_orientation', 'glasses_bbx']] = df[['face_bbx', 'left_eye_bbx', 'right_eye_bbx', 'occluder_bbx', 'occluder_type', 'occluder_degree', 'face_gender', 'face_race', 'face_orientation', 'glasses_bbx']].map(np.array)

# Drop the original 'label' column
df.drop(columns=['label'], inplace=True)

# Save the DataFrame to a CSV file without square brackets
df.to_csv('csv/MAFA_Label_Train.csv', index=False)

# Display the cleaned DataFrame
print("\nCleaned and structured DataFrame:")
print(df.head())

In [ ]:
# Data Structuring and Preprocessing v4

import pandas as pd
import numpy as np
import scipy.io
from PIL import Image
import imagehash
import os
import logging

if not os.path.exists('logs'):
    os.makedirs('logs')

# Load the .mat file
train_data = scipy.io.loadmat(r'C:\Users\alexw\OneDrive\Documents\03_Education\University_Programming\Python\Big_Data\Coursework\Datasets\MAFA\MAFA-Label-Train\LabelTrainAll.mat')
image_dir = r'C:\Users\alexw\OneDrive\Documents\03_Education\University_Programming\Python\Big_Data\Coursework\Datasets\MAFA\MAFA-Label-Train\train-images'

# Convert the data to a DataFrame
df = pd.DataFrame(train_data['label_train'][0])

# Preprocess DataFrame to remove square brackets
df['orgImgName'] = df['orgImgName'].apply(lambda x: x[0])
df['imgName'] = df['imgName'].apply(lambda x: x[0])
df['label'] = df['label'].apply(lambda x: ', '.join(map(str, x[0])))

### Missing Value validation

# Identify missing values
missing_values = df.isnull().sum()
print("Missing Values:")
print(missing_values)

# Fill in the missing values with the mean value.
# df.fillna(df.mean(), inplace=True) # Cannot be done with images, thus we just remove the row completely.
df.dropna(inplace=True)

### Validating the values column to ensure correct number of elements

# Validate the number of elements in the 'label' column
valid_label_count = df['label'].apply(lambda x: len(x.split(', ')) == 21)
missing_values = valid_label_count.value_counts()[True]  # Count missing values
print("\nNumber of elements with expected label array size:", missing_values)
# print("Missing values:", missing_values)

### Double checking for duplicate values - Does nothing with them, just a note at the moment.

logging.basicConfig(filename='logs\duplicate_images.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def find_duplicate_images_2(image_dir):
    """
    Find duplicate images in a directory based on their perceptual hashes and log them.
    """
    # Dictionary to store hashes and corresponding filenames
    hash_dict = {}
    
    # List to store duplicate image filenames
    duplicate_images = []
    
    # Iterate through all files in the directory
    for filename in os.listdir(image_dir):
        # Check if the file is an image
        if filename.endswith((".jpg", ".png")):
            # Get the full file path
            filepath = os.path.join(image_dir, filename)
            
            # Open the image and calculate its hash
            try:
                with Image.open(filepath) as img:
                    hash_val = str(imagehash.average_hash(img))
            except Exception as e:
                logging.error(f"Error processing image {filename}: {e}")
                continue
            
            # Check if the hash already exists in the dictionary
            if hash_val in hash_dict:
                # Add the current filename and the filename already stored in hash_dict
                duplicate_images.append((filename, hash_dict[hash_val]))
                logging.info(f"Duplicate image found: {filename} is a duplicate of {hash_dict[hash_val]}")
            else:
                # Add the hash and filename to the dictionary
                hash_dict[hash_val] = filename
                
    # Log the results
    if duplicate_images:
        logging.info(f"Total {len(duplicate_images)} duplicate images found.")
    else:
        logging.info("No duplicate images found.")

# Call the function to find and print duplicate images
find_duplicate_images_2(image_dir)

### Structure the data and convert into a csv

# Extract bounding box information
label_columns = ['face_x', 'face_y', 'face_w', 'face_h', 'eye1_x', 'eye1_y', 'eye2_x', 'eye2_y', 'occluder_x', 'occluder_y', 'occluder_w', 'occluder_h', 'occ_type', 'occ_degree', 'gender', 'race', 'orientation', 'glasses_x', 'glasses_y', 'glasses_w', 'glasses_h']
df[label_columns] = df['label'].str.split(', ', expand=True)

# Create separate columns for bounding boxes
df['face_bbx'] = df[['face_x', 'face_y', 'face_w', 'face_h']].values.tolist()
df['left_eye_bbx'] = df[['eye1_x', 'eye1_y']].values.tolist()
df['right_eye_bbx'] = df[['eye2_x', 'eye2_y']].values.tolist()
df['occluder_bbx'] = df[['occluder_x', 'occluder_y', 'occluder_w', 'occluder_h']].values.tolist()
df['occluder_type'] = df['occ_type']
df['occluder_degree'] = df['occ_degree']
df['face_gender'] = df['gender']
df['face_race'] = df['race']
df['face_orientation'] = df['orientation']
df['glasses_bbx'] = df[['glasses_x', 'glasses_y', 'glasses_w', 'glasses_h']].values.tolist()

# Drop the original columns
df.drop(columns=label_columns, inplace=True)

# Convert the extracted columns to numeric type
df[['face_bbx', 'left_eye_bbx', 'right_eye_bbx', 'occluder_bbx', 'occluder_type', 'occluder_degree', 'face_gender', 'face_race', 'face_orientation', 'glasses_bbx']] = df[['face_bbx', 'left_eye_bbx', 'right_eye_bbx', 'occluder_bbx', 'occluder_type', 'occluder_degree', 'face_gender', 'face_race', 'face_orientation', 'glasses_bbx']].map(np.array)

# Drop the original 'label' column
df.drop(columns=['label'], inplace=True)

# Save the DataFrame to a CSV file without square brackets
df.to_csv('csv/MAFA_Label_Train.csv', index=False)

# Display the cleaned DataFrame
print("\nCleaned and structured DataFrame:")
print(df.head())

In [ ]:
# Data Structuring and Preprocessing v5

import pandas as pd
import numpy as np
import scipy.io
from PIL import Image
import imagehash
import os
import logging

if not os.path.exists('logs'):
    os.makedirs('logs')

# Load the .mat file
train_data = scipy.io.loadmat(r'C:\Users\alexw\OneDrive\Documents\03_Education\University_Programming\Python\Big_Data\Coursework\Datasets\MAFA\MAFA-Label-Train\LabelTrainAll.mat')
image_dir = r'C:\Users\alexw\OneDrive\Documents\03_Education\University_Programming\Python\Big_Data\Coursework\Datasets\MAFA\MAFA-Label-Train\train-images'

# Convert the data to a DataFrame
df = pd.DataFrame(train_data['label_train'][0])

# Preprocess DataFrame to remove square brackets
df['orgImgName'] = df['orgImgName'].apply(lambda x: x[0])
df['imgName'] = df['imgName'].apply(lambda x: x[0])
df['label'] = df['label'].apply(lambda x: ', '.join(map(str, x[0])))

### Missing Value validation

# Identify missing values
missing_values = df.isnull().sum()
print("Missing Values:")
print(missing_values)

# Fill in the missing values with the mean value.
# df.fillna(df.mean(), inplace=True) # Cannot be done with images, thus we just remove the row completely.
df.dropna(inplace=True)

### Validating the values column to ensure correct number of elements

# Validate the number of elements in the 'label' column
valid_label_count = df['label'].apply(lambda x: len(x.split(', ')) == 21)
missing_values = valid_label_count.value_counts()[True]  # Count missing values
print("\nNumber of elements with expected label array size:", missing_values)
# print("Missing values:", missing_values)

### Double checking for duplicate values - Does nothing with them, just a note at the moment.

logging.basicConfig(filename='logs\duplicate_images.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def find_duplicate_images(image_dir):
    """
    Find duplicate images in a directory based on their perceptual hashes and log them.
    """
    # Dictionary to store hashes and corresponding filenames
    hash_dict = {}
    
    # List to store duplicate image filenames
    duplicate_images = []
    
    # Iterate through all files in the directory
    for filename in os.listdir(image_dir):
        # Check if the file is an image
        if filename.endswith((".jpg", ".png")):
            # Get the full file path
            filepath = os.path.join(image_dir, filename)
            
            # Open the image and calculate its hash
            try:
                with Image.open(filepath) as img:
                    hash_val = str(imagehash.average_hash(img))
            except Exception as e:
                logging.error(f"Error processing image {filename}: {e}")
                continue
            
            # Check if the hash already exists in the dictionary
            if hash_val in hash_dict:
                # Add the current filename and the filename already stored in hash_dict
                duplicate_images.append((filename, hash_dict[hash_val]))
                logging.info(f"Duplicate image found: {filename} is a duplicate of {hash_dict[hash_val]}")
            else:
                # Add the hash and filename to the dictionary
                hash_dict[hash_val] = filename
                
    # Log the results
    if duplicate_images:
        logging.info(f"Total {len(duplicate_images)} duplicate images found.")
    else:
        logging.info("No duplicate images found.")

# Call the function to find and print duplicate images
find_duplicate_images(image_dir)

### Structure the data and convert into a csv

# Extract bounding box information
label_columns = ['face_x', 'face_y', 'face_w', 'face_h', 'eye1_x', 'eye1_y', 'eye2_x', 'eye2_y', 'occluder_x', 'occluder_y', 'occluder_w', 'occluder_h', 'occ_type', 'occ_degree', 'gender', 'race', 'orientation', 'glasses_x', 'glasses_y', 'glasses_w', 'glasses_h']
df[label_columns] = df['label'].str.split(', ', expand=True)

# Create separate columns for bounding boxes
df['face_bbx'] = df.apply(lambda row: ', '.join(row[['face_x', 'face_y', 'face_w', 'face_h']].astype(str)), axis=1)
df['left_eye_bbx'] = df.apply(lambda row: ', '.join(row[['eye1_x', 'eye1_y']].astype(str)), axis=1)
df['right_eye_bbx'] = df.apply(lambda row: ', '.join(row[['eye2_x', 'eye2_y']].astype(str)), axis=1)
df['occluder_bbx'] = df.apply(lambda row: ', '.join(row[['occluder_x', 'occluder_y', 'occluder_w', 'occluder_h']].astype(str)), axis=1)
df['occluder_type'] = df['occ_type']
df['occluder_degree'] = df['occ_degree']
# df['face_gender'] = df['gender'] # Removing these columns is a form of dimensionality reduction
# df['face_race'] = df['race']
# df['face_orientation'] = df['orientation']
# df['glasses_bbx'] = df.apply(lambda row: ', '.join(row[['glasses_x', 'glasses_y', 'glasses_w', 'glasses_h']].astype(str)), axis=1)

# Drop the original columns
df.drop(columns=label_columns, inplace=True)

# Convert the extracted columns to numeric type
df[['face_bbx', 'left_eye_bbx', 'right_eye_bbx', 'occluder_bbx', 'occluder_type', 'occluder_degree', 'face_gender', 'face_race', 'face_orientation', 'glasses_bbx']] = df[['face_bbx', 'left_eye_bbx', 'right_eye_bbx', 'occluder_bbx', 'occluder_type', 'occluder_degree', 'face_gender', 'face_race', 'face_orientation', 'glasses_bbx']].map(np.array)

# Drop the original 'label' column
df.drop(columns=['label'], inplace=True)

# Save the DataFrame to a CSV file without square brackets
df.to_csv('csv/MAFA_Label_Train.csv', index=False)

# Display the cleaned DataFrame
print("\nCleaned and structured DataFrame:")
print(df.head())

In [ ]:
# Used - Data Structuring and Preprocessing v6

if not os.path.exists('logs'):
    os.makedirs('logs')

# Load the .mat file
train_data = scipy.io.loadmat(r'C:\Users\alexw\OneDrive\Documents\03_Education\University_Programming\Python\Big_Data\Coursework\Datasets\MAFA\MAFA-Label-Train\LabelTrainAll.mat')
image_dir = r'C:\Users\alexw\OneDrive\Documents\03_Education\University_Programming\Python\Big_Data\Coursework\Datasets\MAFA\MAFA-Label-Train\train-images'

# Convert the data to a DataFrame
df = pd.DataFrame(train_data['label_train'][0])

# Preprocess DataFrame to remove square brackets
df['orgImgName'] = df['orgImgName'].apply(lambda x: x[0])
df['imgName'] = df['imgName'].apply(lambda x: x[0])
df['label'] = df['label'].apply(lambda x: ', '.join(map(str, x[0])))

### Missing Value validation

# Identify missing values
missing_values = df.isnull().sum()
print("Missing Values:")
print(missing_values)

# Fill in the missing values with the mean value.
# df.fillna(df.mean(), inplace=True) # Cannot be done with images, thus we just remove the row completely.
df.dropna(inplace=True)

### Validating the values column to ensure correct number of elements

# Validate the number of elements in the 'label' column
valid_label_count = df['label'].apply(lambda x: len(x.split(', ')) == 21)
missing_values = valid_label_count.value_counts()[True]  # Count missing values
print("\nNumber of elements with expected label array size:", missing_values)
# print("Missing values:", missing_values)

### Double checking for duplicate values - Does nothing with them, just a note at the moment.

logging.basicConfig(filename='logs\duplicate_images.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def find_duplicate_images(image_dir):
    """
    Find duplicate images in a directory based on their perceptual hashes and log them.
    """
    # Dictionary to store hashes and corresponding filenames
    hash_dict = {}
    
    # List to store duplicate image filenames
    duplicate_images = []
    
    # Iterate through all files in the directory
    for filename in os.listdir(image_dir):
        # Check if the file is an image
        if filename.endswith((".jpg", ".png")):
            # Get the full file path
            filepath = os.path.join(image_dir, filename)
            
            # Open the image and calculate its hash
            try:
                with Image.open(filepath) as img:
                    hash_val = str(imagehash.average_hash(img))
            except Exception as e:
                logging.error(f"Error processing image {filename}: {e}")
                continue
            
            # Check if the hash already exists in the dictionary
            if hash_val in hash_dict:
                # Add the current filename and the filename already stored in hash_dict
                duplicate_images.append((filename, hash_dict[hash_val]))
                logging.info(f"Duplicate image found: {filename} is a duplicate of {hash_dict[hash_val]}")
            else:
                # Add the hash and filename to the dictionary
                hash_dict[hash_val] = filename
                
    # Log the results
    if duplicate_images:
        logging.info(f"Total {len(duplicate_images)} duplicate images found.")
    else:
        logging.info("No duplicate images found.")

# Call the function to find and print duplicate images
# find_duplicate_images(image_dir)

### Structure the data and convert into a csv

# Extract bounding box information
label_columns = ['face_x', 'face_y', 'face_w', 'face_h', 'eye1_x', 'eye1_y', 'eye2_x', 'eye2_y', 'occluder_x', 'occluder_y', 'occluder_w', 'occluder_h', 'occ_type', 'occ_degree', 'gender', 'race', 'orientation', 'glasses_x', 'glasses_y', 'glasses_w', 'glasses_h']
df[label_columns] = df['label'].str.split(', ', expand=True)

# Convert the bounding box coordinates from strings to floats
for col in label_columns:
    df[col] = df[col].astype(float)

# Group bounding box coordinates under their respective columns
df['face_bbx'] = df.apply(lambda row: ', '.join(map(str, row[['face_x', 'face_y', 'face_w', 'face_h']])), axis=1)
df['left_eye_pos'] = df.apply(lambda row: ', '.join(map(str, row[['eye1_x', 'eye1_y']])), axis=1)
df['right_eye_pos'] = df.apply(lambda row: ', '.join(map(str, row[['eye2_x', 'eye2_y']])), axis=1)
df['occluder_bbx'] = df.apply(lambda row: ', '.join(map(str, row[['occluder_x', 'occluder_y', 'occluder_w', 'occluder_h']])), axis=1)
df['occluder_type'] = df['occ_type']
df['occluder_degree'] = df['occ_degree']
df['face_gender'] = df['gender'] # Removing these columns is a form of dimensionality reduction
df['face_race'] = df['race']
df['face_orientation'] = df['orientation']
df['glasses_bbx'] = df.apply(lambda row: ', '.join(map(str, row[['glasses_x', 'glasses_y', 'glasses_w', 'glasses_h']])), axis=1)

# Drop the original columns
df.drop(columns=label_columns, inplace=True)

# Drop the original 'label' column
df.drop(columns=['label'], inplace=True)

# Save the DataFrame to a CSV file
df.to_csv('csv/MAFA_Training_Data_Structured.csv', index=False)

# Display the cleaned DataFrame
print("\nCleaned and structured DataFrame:")
print(df.head())

Preprocessing - Image Augmentation

In [ ]:
# Basic Augmentation - Rotation, Blue, Flipped, and brightness adjustment. No bbx.

import cv2
import numpy as np
import matplotlib.pyplot as plt

def augment_and_plot(row):
    # Load the image
    img_name = row['imgName']
    img = cv2.imread(image_path + '\\' + img_name)
    
    # Convert image to RGB for displaying with matplotlib
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Plot the original image
    plt.figure(figsize=(15, 6))
    plt.subplot(1, 5, 1)
    plt.imshow(img_rgb)
    plt.title('Original')
    plt.axis('off')
    
    # Augmentation 1: Rotation
    rotation_angle = 30
    rotation_matrix = cv2.getRotationMatrix2D((img.shape[1] / 2, img.shape[0] / 2), rotation_angle, 1)
    rotated_img = cv2.warpAffine(img, rotation_matrix, (img.shape[1], img.shape[0]))
    plt.subplot(1, 5, 2)
    plt.imshow(cv2.cvtColor(rotated_img, cv2.COLOR_BGR2RGB))
    plt.title('Rotated')
    plt.axis('off')
    
    # Augmentation 2: Blur
    blurred_img = cv2.GaussianBlur(img, (15, 15), 0)
    plt.subplot(1, 5, 3)
    plt.imshow(cv2.cvtColor(blurred_img, cv2.COLOR_BGR2RGB))
    plt.title('Blurred')
    plt.axis('off')
    
    # Augmentation 3: Flipping
    flipped_img = cv2.flip(img, 1)  # 1 for horizontal flip
    plt.subplot(1, 5, 4)
    plt.imshow(cv2.cvtColor(flipped_img, cv2.COLOR_BGR2RGB))
    plt.title('Flipped')
    plt.axis('off')
    
    # Augmentation 4: Brightness adjustment
    brightness_adjusted_img = cv2.convertScaleAbs(img, alpha=2.0, beta=50)
    plt.subplot(1, 5, 5)
    plt.imshow(cv2.cvtColor(brightness_adjusted_img, cv2.COLOR_BGR2RGB))
    plt.title('Brightness Adjusted')
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()

# Testing the function
# Assume `df` is your DataFrame containing image data
row_index = 0  # Choose a specific row for testing
row = df.iloc[row_index]
augment_and_plot(row)


In [ ]:
# Adds bbx to the one above. Rotated bbx doesn't work - misplaced.

def augment_and_plot(row):
    # Load the image
    img_name = row['imgName']
    img = cv2.imread(image_path + '\\' + img_name)
    
    # Convert image to RGB for displaying with matplotlib
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Plot the original image with bounding box
    plt.figure(figsize=(15, 6))
    plt.subplot(1, 5, 1)
    plt.imshow(img_rgb)
    plt.title('Original')
    plt.axis('off')
    draw_bounding_box(row, img.shape[1], img.shape[0])
    
    # Augmentation 1: Rotation
    rotation_angle = 30
    rotation_matrix = cv2.getRotationMatrix2D((img.shape[1] / 2, img.shape[0] / 2), rotation_angle, 1)
    rotated_img = cv2.warpAffine(img, rotation_matrix, (img.shape[1], img.shape[0]))
    plt.subplot(1, 5, 2)
    plt.imshow(cv2.cvtColor(rotated_img, cv2.COLOR_BGR2RGB))
    plt.title('Rotated')
    plt.axis('off')
    draw_bounding_box(row, img.shape[1], img.shape[0], rotation_angle)
    
    # Augmentation 2: Blur
    blurred_img = cv2.GaussianBlur(img, (15, 15), 0)
    plt.subplot(1, 5, 3)
    plt.imshow(cv2.cvtColor(blurred_img, cv2.COLOR_BGR2RGB))
    plt.title('Blurred')
    plt.axis('off')
    draw_bounding_box(row, img.shape[1], img.shape[0])
    
    # Augmentation 3: Flipping
    flipped_img = cv2.flip(img, 1)  # 1 for horizontal flip
    plt.subplot(1, 5, 4)
    plt.imshow(cv2.cvtColor(flipped_img, cv2.COLOR_BGR2RGB))
    plt.title('Flipped')
    plt.axis('off')
    draw_bounding_box(row, img.shape[1], img.shape[0], flip=True)
    
    # Augmentation 4: Brightness adjustment
    brightness_adjusted_img = cv2.convertScaleAbs(img, alpha=2.0, beta=50)
    plt.subplot(1, 5, 5)
    plt.imshow(cv2.cvtColor(brightness_adjusted_img, cv2.COLOR_BGR2RGB))
    plt.title('Brightness Adjusted')
    plt.axis('off')
    draw_bounding_box(row, img.shape[1], img.shape[0])
    
    plt.tight_layout()
    plt.show()

# Function to draw bounding boxes
def draw_bounding_box(row, img_width, img_height, rotation_angle=0, flip=False):
    face_x, face_y, face_w, face_h = row['face_x'], row['face_y'], row['face_w'], row['face_h']
    occluder_x, occluder_y, occluder_w, occluder_h = row['occluder_x'], row['occluder_y'], row['occluder_w'], row['occluder_h']
    glasses_x, glasses_y, glasses_w, glasses_h = row['glasses_x'], row['glasses_y'], row['glasses_w'], row['glasses_h']
    
    # Adjust occluder coordinates relative to the face bounding box
    occluder_x += face_x
    occluder_y += face_y
    
    if rotation_angle != 0:
        # Rotate bounding boxes
        face_x, face_y = rotate_point(face_x, face_y, img_width / 2, img_height / 2, rotation_angle)
        occluder_x, occluder_y = rotate_point(occluder_x, occluder_y, img_width / 2, img_height / 2, rotation_angle)
        glasses_x, glasses_y = rotate_point(glasses_x, glasses_y, img_width / 2, img_height / 2, rotation_angle)
    
    if flip:
        # Flip bounding boxes
        face_x = img_width - face_x - face_w
        occluder_x = img_width - occluder_x - occluder_w
        glasses_x = img_width - glasses_x - glasses_w
    
    plt.gca().add_patch(patches.Rectangle((face_x, face_y), face_w, face_h, linewidth=2, edgecolor='r', facecolor='none'))
    plt.gca().add_patch(patches.Rectangle((occluder_x, occluder_y), occluder_w, occluder_h, linewidth=2, edgecolor='b', facecolor='none'))
    plt.gca().add_patch(patches.Rectangle((glasses_x, glasses_y), glasses_w, glasses_h, linewidth=2, edgecolor='g', facecolor='none'))


# Function to rotate a point around an origin
def rotate_point(x, y, cx, cy, angle):
    angle_rad = np.deg2rad(angle)
    x_new = (x - cx) * np.cos(angle_rad) - (y - cy) * np.sin(angle_rad) + cx
    y_new = (x - cx) * np.sin(angle_rad) + (y - cy) * np.cos(angle_rad) + cy
    return x_new, y_new

# Testing the function
# Assume `df` is your DataFrame containing image data
row_index = 0  # Choose a specific row for testing
row = df.iloc[row_index]
augment_and_plot(row)


In [ ]:
# THIS IS THE CODE TO BLUR AND CAN BE USED FOR AUGMENTATION - Both skimage and cv2 have been used.

import cv2
import scipy.io as sio
import os
import pandas as pd
import matplotlib.pyplot as plt
from skimage import io, filters

# Load bounding box annotations from the .mat file
mat_file = r'C:\Users\alexw\OneDrive\Documents\03_Education\University\University_Programming\Python\Big_Data\Coursework\Datasets\WIDERFACE\wider_face_split\wider_face_split\wider_face_train.mat'
annotations = sio.loadmat(mat_file)

# Read images from the 'WIDER_train' directory
image_dir = 'C:\\Users\\alexw\\OneDrive\\Documents\\03_Education\\University\\University_Programming\\Python\Big_Data\\Coursework\Datasets\WIDERFACE\WIDER_train\WIDER_train\\images\\0--Parade\\0_Parade_marchingband_1_849.jpg'
image_filenames = annotations['file_list']

# Load the image using skimage
img = io.imread(image_dir)

# Apply blur using skimage (adjust the sigma parameter for the desired blur strength)
blurred_img_skimage = filters.gaussian(img, sigma=9)

# Apply blur using OpenCV (adjust the (5, 5) kernel size for the desired blur strength)
blurred_img_cv2 = cv2.blur(img, (15, 15))  # You can adjust (15, 15) to another size

# Display the original and blurred images using matplotlib
fig, axes = plt.subplots(1, 3, figsize=(12, 4))

axes[0].imshow(img)
axes[0].set_title('Original')
axes[0].axis('off')

axes[1].imshow(blurred_img_skimage)
axes[1].set_title('Blurred (skimage)')
axes[1].axis('off')

axes[2].imshow(blurred_img_cv2)
axes[2].set_title('Blurred (OpenCV)')
axes[2].axis('off')

plt.show()


In [ ]:
# THIS IS THE CODE TO ROTATE AND CAN BE USED FOR AUGMENTATION - Both skimage and cv2 have been used.

import cv2
import scipy.io as sio
import os
import pandas as pd
import matplotlib.pyplot as plt
from skimage import io, filters, transform
import numpy as np

# Load bounding box annotations from the .mat file
mat_file = r'C:\Users\alexw\OneDrive\Documents\03_Education\University\University_Programming\Python\Big_Data\Coursework\Datasets\WIDERFACE\wider_face_split\wider_face_split\wider_face_train.mat'
annotations = sio.loadmat(mat_file)

# Read images from the 'WIDER_train' directory
image_dir = 'C:\\Users\\alexw\\OneDrive\\Documents\\03_Education\\University\\University_Programming\\Python\Big_Data\\Coursework\Datasets\WIDERFACE\WIDER_train\WIDER_train\\images\\0--Parade\\0_Parade_marchingband_1_849.jpg'
image_filenames = annotations['file_list']

# Load the image using skimage
img = io.imread(image_dir)

# Generate a random rotation angle between -30 and 30 degrees
# rotation_angle = 30
rotation_angle = np.random.uniform(-360, 360)

rotated_img_skimage = transform.rotate(img, rotation_angle, resize=False)

# Rotate the image using OpenCV
rows, cols, _ = img.shape
rotation_matrix = cv2.getRotationMatrix2D((cols / 2, rows / 2), rotation_angle, 1)
rotated_img_cv2 = cv2.warpAffine(img, rotation_matrix, (cols, rows))

# Display the original and rotated images using matplotlib
fig, axes = plt.subplots(1, 3, figsize=(12, 4))

axes[0].imshow(img)
axes[0].set_title('Original')
axes[0].axis('off')

axes[1].imshow(rotated_img_skimage)
axes[1].set_title(f'{rotation_angle:.2f} degrees (Skimage)')
axes[1].axis('off')

axes[2].imshow(rotated_img_cv2)
axes[2].set_title(f'{rotation_angle:.2f} degrees (OpenCV)')
axes[2].axis('off')

plt.show()


Functions

In [ ]:
# Function for finding duplicate images within the training image set using image hashing. 

from PIL import Image
import imagehash
import os

def find_duplicates_images(image_dir):
    """
    Find duplicate images in a directory based on their perceptual hashes and print them.
    """
    # Dictionary to store hashes and corresponding filenames
    hash_dict = {}
    
    # List to store duplicate image filenames
    duplicate_images = []
    
    # Iterate through all files in the directory
    for filename in os.listdir(image_dir):
        # Check if the file is an image
        if filename.endswith((".jpg", ".png")):
            # Get the full file path
            filepath = os.path.join(image_dir, filename)
            
            # Open the image and calculate its hash
            with Image.open(filepath) as img:
                hash_val = str(imagehash.average_hash(img))
            
            # Check if the hash already exists in the dictionary
            if hash_val in hash_dict:
                # Add the current filename and the filename already stored in hash_dict
                duplicate_images.append((filename, hash_dict[hash_val]))
            else:
                # Add the hash and filename to the dictionary
                hash_dict[hash_val] = filename
                
    # Print the duplicate images
    if duplicate_images:
        print("Duplicate images found:")
        for img1, img2 in duplicate_images:
            print(f"{img2} is a duplicate of {img1}")
    else:
        print("No duplicate images found.")

# Directory containing images
image_dir = r'C:\Users\alexw\OneDrive\Documents\03_Education\University\University_Programming\Python\Big_Data\Coursework\Datasets\MAFA\train-images\images'

# Call the function to find and print duplicate images
find_duplicates_images(image_dir)


In [ ]:
# This function is used for image hashing. Ensuring that there are no duplicate images in the dataset. DELETES THE IMAGE IN THE FOLDER SO WATCH OUT.

from PIL import Image
import imagehash
import os

# Function to find duplicate images in a directory
def find_duplicate_images(image_dir):
    # Dictionary to store hashes and corresponding filenames
    hash_dict = {}
    
    # List to store duplicate image filenames
    duplicate_images = []
    
    # Iterate through all files in the directory
    for filename in os.listdir(image_dir):
        # Check if the file is an image
        if filename.endswith(".jpg") or filename.endswith(".png"):
            # Get the full file path
            filepath = os.path.join(image_dir, filename)
            
            # Open the image and calculate its hash
            with Image.open(filepath) as img:
                hash_val = str(imagehash.average_hash(img))
            
            # Check if the hash already exists in the dictionary
            if hash_val in hash_dict:
                # Add the filename to the list of duplicates
                duplicate_images.append(filename)
            else:
                # Add the hash and filename to the dictionary
                hash_dict[hash_val] = filename
                
    # Iterate through the list of duplicate images
    for duplicate in duplicate_images:
        # Get the full file path of the duplicate image
        duplicate_path = os.path.join(image_dir, duplicate)
        
        # Delete the duplicate image
        os.remove(duplicate_path)
        
        print(f"Deleted duplicate image: {duplicate}")

# Directory containing the images
image_dir = r'C:\Users\alexw\OneDrive\Documents\03_Education\University\University_Programming\Python\Big_Data\Coursework\Datasets\MAFA\images'
# find_duplicate_images(image_dir)


Working Models

CNN

In [ ]:
# Successfully trains model, and then predicts the test images and displays the results with the image

import pandas as pd
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

# Load the CSV file
image_labels = pd.read_csv(r'C:\Users\alexw\OneDrive\Documents\03_Education\University_Programming\Python\Big_Data\Coursework\Masked_Face\csv\MAFA_Training_Data_Unstructured.csv')
path_to_images = r"C:\Users\alexw\OneDrive\Documents\03_Education\University_Programming\Python\Big_Data\Coursework\Datasets\MAFA\MAFA-Label-Train\train-images"

# Add mask_label column - defines whether the occluder in the image is a simple mask, complex mask or whether it is a human bodypart occluding.
image_labels['mask_label'] = np.where((image_labels['occluder_type'] == 1) | (image_labels['occluder_type'] == 2), 1, 0)

# Define image size
img_size = 100

# Load and preprocess images
def preprocess_image(img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (img_size, img_size))
    img = img / 255.0 # Normalize pixel values
    return img

# Load images and labels
X = np.array([preprocess_image(os.path.join(path_to_images, img_name)) for img_name in image_labels['imgName']])
y = to_categorical(image_labels['mask_label'])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define image augmentation parameters
datagen = ImageDataGenerator(
    rotation_range=20, # Rotate images by 20 degrees
    width_shift_range=0.1, # Shift images horizontally by 10%
    height_shift_range=0.1, # Shift images vertically by 10%
    shear_range=0.2, # Shear angle in counter-clockwise direction
    zoom_range=0.2, # Zoom range from 80% to 120%
    horizontal_flip=True, # Randomly flip images horizontally
    fill_mode='nearest' # Fill mode for filling in newly created pixels
)

# Create augmented data generator
augmented_datagen = datagen.flow(X_train, y_train, batch_size=32)

# Define the model architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax') # 2 output nodes for binary classification (with mask or without mask)
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit_generator(augmented_datagen, epochs=10, validation_data=(X_test, y_test))

# Evaluate the model on test data
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)
accuracy = accuracy_score(y_test_classes, y_pred_classes)
precision = precision_score(y_test_classes, y_pred_classes)
recall = recall_score(y_test_classes, y_pred_classes)
f1 = f1_score(y_test_classes, y_pred_classes)
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

# Load the test data
test_data = pd.read_csv(r'C:\Users\alexw\OneDrive\Documents\03_Education\University_Programming\Python\Big_Data\Coursework\Masked_Face\csv\MAFA_test_data.csv')
test_images_path = r"C:\Users\alexw\OneDrive\Documents\03_Education\University_Programming\Python\Big_Data\Coursework\Datasets\MAFA\MAFA-Label-Test\test-images"

# Display test images with face mask caption
for _, row in test_data.iterrows():
    img_name = row['imgName']
    img_path = os.path.join(test_images_path, img_name)
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    face_x, face_y, face_w, face_h = row[['face_x', 'face_y', 'face_w', 'face_h']]
    occluder_type = row['occluder_type']

    # Check if the occluder type is a mask
    is_mask = occluder_type == 1 or occluder_type == 2

    # Draw a bounding box around the face
    cv2.rectangle(img, (int(face_x), int(face_y)), (int(face_x + face_w), int(face_y + face_h)), (0, 255, 0), 2)

    # Add a caption indicating if the person is wearing a mask or not
    caption = "Wearing a face mask" if is_mask else "Not wearing a face mask"
    
    plt.figure(figsize=(10, 8))
    plt.title(caption)
    plt.imshow(img)
    plt.show()


In [ ]:
# WORKING CNN - Accuracy of 0.9223 with image resizing but no augmentation.

import pandas as pd
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.utils import to_categorical

# Load the CSV file
image_labels = pd.read_csv(r'C:\Users\alexw\OneDrive\Documents\03_Education\University_Programming\Python\Big_Data\Coursework\Masked_Face\csv\MAFA_Training_Data_Unstructured.csv')
path_to_images = r"C:\Users\alexw\OneDrive\Documents\03_Education\University_Programming\Python\Big_Data\Coursework\Datasets\MAFA\MAFA-Label-Train\train-images"

# Add mask_label column
image_labels['mask_label'] = np.where((image_labels['occluder_type'] == 1) | (image_labels['occluder_type'] == 2), 1, 0)

# Define image size
IMG_SIZE = 100 # 224 x 224 ideal image size but too large to create an array.

# Load and preprocess images
def preprocess_image(img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = img / 255.0  # Normalize pixel values
    return img

# Load images and labels
X = np.array([preprocess_image(os.path.join(path_to_images, img_name)) for img_name in image_labels['imgName']])
y = to_categorical(image_labels['mask_label'])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')  # 2 output nodes for binary classification (with mask or without mask)
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model on test data
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

accuracy = accuracy_score(y_test_classes, y_pred_classes)
precision = precision_score(y_test_classes, y_pred_classes)
recall = recall_score(y_test_classes, y_pred_classes)
f1 = f1_score(y_test_classes, y_pred_classes)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

In [ ]:
# WORKING CNN - Accuracy of 0.9223 with image augmentation - Accuracy seems to have converged upon 0.922 thus further processing or augmentation will not improve.

import pandas as pd
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator

# Load the CSV file
image_labels = pd.read_csv(r'C:\Users\alexw\OneDrive\Documents\03_Education\University_Programming\Python\Big_Data\Coursework\Masked_Face\csv\MAFA_Training_Data_Unstructured.csv')
path_to_images = r"C:\Users\alexw\OneDrive\Documents\03_Education\University_Programming\Python\Big_Data\Coursework\Datasets\MAFA\MAFA-Label-Train\train-images"

# Add mask_label column - defines whether the occluder in the image is a simple mask, complex mask or whether it is a human bodypart occluding.
image_labels['mask_label'] = np.where((image_labels['occluder_type'] == 1) | (image_labels['occluder_type'] == 2), 1, 0)

# Define image size
img_size = 100

# Load and preprocess images
def preprocess_image(img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (img_size, img_size))
    img = img / 255.0  # Normalize pixel values
    return img

# Load images and labels
X = np.array([preprocess_image(os.path.join(path_to_images, img_name)) for img_name in image_labels['imgName']])
y = to_categorical(image_labels['mask_label'])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define image augmentation parameters - Doesn't really do anything and the model tends to converge around 0.9220
datagen = ImageDataGenerator(
    rotation_range=20,      # Rotate images by 20 degrees
    width_shift_range=0.1,  # Shift images horizontally by 10%
    height_shift_range=0.1, # Shift images vertically by 10%
    shear_range=0.2,        # Shear angle in counter-clockwise direction
    zoom_range=0.2,         # Zoom range from 80% to 120%
    horizontal_flip=True,   # Randomly flip images horizontally
    fill_mode='nearest'     # Fill mode for filling in newly created pixels
)

# Create augmented data generator
augmented_datagen = datagen.flow(X_train, y_train, batch_size=32)

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')  # 2 output nodes for binary classification (with mask or without mask)
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit_generator(augmented_datagen, epochs=10, validation_data=(X_test, y_test))

# Evaluate the model on test data
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

accuracy = accuracy_score(y_test_classes, y_pred_classes)
precision = precision_score(y_test_classes, y_pred_classes)
recall = recall_score(y_test_classes, y_pred_classes)
f1 = f1_score(y_test_classes, y_pred_classes)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')




SVM

In [ ]:
# Working SVM Model...Accuracy 0.8765 on 10% of the data.

import pandas as pd
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder

# Load the CSV file
image_labels = pd.read_csv(r'C:\Users\alexw\OneDrive\Documents\03_Education\University_Programming\Python\Big_Data\Coursework\Masked_Face\csv\MAFA_Training_Data_Unstructured.csv')

path_to_images = r"C:\Users\alexw\OneDrive\Documents\03_Education\University_Programming\Python\Big_Data\Coursework\Datasets\MAFA\MAFA-Label-Train\train-images"

# Add mask_label column
image_labels['mask_label'] = np.where((image_labels['occluder_type'] == 1) | (image_labels['occluder_type'] == 2), 1, 0)

# Define image size
img_size = 100

# Load and preprocess images
def preprocess_image(img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (img_size, img_size))
    img = img.flatten() / 255.0  # Flatten and normalize pixel values
    return img

# Load images and labels
X = np.array([preprocess_image(os.path.join(path_to_images, img_name)) for img_name in image_labels['imgName']])
y = image_labels['mask_label'].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reduce training data size (e.g. 10% of the data)
reduce_factor = 0.1
num_samples_to_train = int(len(X_train) * reduce_factor)
X_train_reduced = X_train[:num_samples_to_train]
y_train_reduced = y_train[:num_samples_to_train]

# Create and train the SVM model
svm_model = SVC(kernel='linear', probability=True)
svm_model.fit(X_train_reduced, y_train_reduced)

# Evaluate the model on test data
y_pred = svm_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')


In [2]:
# Working SVM Model...Accuracy 0.534 on unstructured raw data. Confusion matrix doesn't work.

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import svm

def confusionM(y_true, y_predict, target_names):
    # Function for visualization
    cMatrix = confusion_matrix(y_true, y_predict)
    df_cm = pd.DataFrame(cMatrix, index=target_names, columns=target_names)
    plt.figure(figsize=(6, 4))
    sns.heatmap(df_cm, annot=True, fmt="d", cmap="Blues")
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.title('Confusion Matrix')
    plt.show()

# Load your dataset
data = pd.read_csv(r"C:\Users\alexw\OneDrive\Documents\03_Education\University_Programming\Python\Big_Data\Coursework\Masked_Face\csv\MAFA_Training_Data_Unstructured.csv")

# Assuming 'occluder_type' represents whether someone is wearing a face mask or not (1 = wearing, 2 = not wearing)
# Create features X and target y
X = data[['face_x', 'face_y', 'face_w', 'face_h', 'eye1_x', 'eye1_y', 'eye2_x', 'eye2_y', 'occluder_x', 'occluder_y', 'occluder_w', 'occluder_h']]
y = data['occluder_type']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_true = train_test_split(X, y, test_size=0.1, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize SVM classifier
svc = svm.SVC(kernel='linear')

# Train the SVM model
svc.fit(X_train_scaled, y_train)

# Test the SVM model
accuracy = svc.score(X_test_scaled, y_true)
print("Accuracy:", accuracy)

# Predict on test data
y_predict = svc.predict(X_test_scaled)
print("Predictions:", y_predict)

# Visualize confusion matrix
# confusionM(y_true, y_predict, ['Not Wearing', 'Wearing'])

Accuracy: 0.5343894899536321
Predictions: [1. 1. 2. ... 2. 1. 1.]


In [ ]:
# Working SVM Model...Accuracy 0.527, very basic model...Might be a good one to showcase.

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.utils import shuffle

def confusionM(y_true, y_predict, target_names):
    # Function for visualization
    cMatrix = confusion_matrix(y_true, y_predict)
    df_cm = pd.DataFrame(cMatrix, index=target_names, columns=target_names)
    plt.figure(figsize=(6, 4))
    sns.heatmap(df_cm, annot=True, fmt="d", cmap="Blues")
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.title('Confusion Matrix')
    plt.show()

# Load your dataset
data = pd.read_csv(r"C:\Users\alexw\OneDrive\Documents\03_Education\University_Programming\Python\Big_Data\Coursework\Masked_Face\csv\MAFA_Training_Data_Unstructured.csv")

# Assuming 'occluder_type' represents whether someone is wearing a face mask or not (1 = wearing, 2 = not wearing)
# Create features X and target y
X = data[['face_x', 'face_y', 'face_w', 'face_h', 'eye1_x', 'eye1_y', 'eye2_x', 'eye2_y', 'occluder_x', 'occluder_y', 'occluder_w', 'occluder_h']]
y = data['occluder_type']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize SVM classifier
svc = SVC(kernel='rbf', C=10, gamma='auto', class_weight='balanced')

# Train the SVM model
svc.fit(X_train_scaled, y_train)

# Predict on test data
y_predict = svc.predict(X_test_scaled)

# Evaluate the model
accuracy = svc.score(X_test_scaled, y_test)
print("Accuracy:", accuracy)

# Visualize confusion matrix
# confusionM(y_test, y_predict, ['Wearing', 'Not Wearing'])  # Adjust the target names accordingly


In [3]:
# Working SVM Model...Accuracy 0.536. Improving... Used the resized dataset.

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.utils import shuffle

def confusionM(y_true, y_predict, target_names):
    # Function for visualization
    cMatrix = confusion_matrix(y_true, y_predict)
    df_cm = pd.DataFrame(cMatrix, index=target_names, columns=target_names)
    plt.figure(figsize=(6, 4))
    sns.heatmap(df_cm, annot=True, fmt="d", cmap="Blues")
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.title('Confusion Matrix')
    plt.show()

# Load your dataset
data = pd.read_csv(r"C:\Users\alexw\OneDrive\Documents\03_Education\University_Programming\Python\Big_Data\Coursework\Masked_Face\csv\MAFA_Training_Data_Unstructured_Resized.csv")

# Assuming 'occluder_type' represents whether someone is wearing a face mask or not (1 = wearing, 2 = not wearing)
# Create features X and target y
X = data[['face_x', 'face_y', 'face_w', 'face_h', 'eye1_x', 'eye1_y', 'eye2_x', 'eye2_y', 'occluder_x', 'occluder_y', 'occluder_w', 'occluder_h']]
y = data['occluder_type']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize SVM classifier
svc = SVC(kernel='rbf', C=10, gamma='auto', class_weight='balanced')

# Train the SVM model
svc.fit(X_train_scaled, y_train)

# Predict on test data
y_predict = svc.predict(X_test_scaled)

# Evaluate the model
accuracy = svc.score(X_test_scaled, y_test)
print("Accuracy:", accuracy)

# Visualize confusion matrix
# confusionM(y_test, y_predict, ['Wearing', 'Not Wearing'])  # Adjust the target names accordingly


Accuracy: 0.535935085007728


Might Come in Useful but not wanted right now

In [ ]:
# Resizes images and adjusts labels accordingly v2...Demonstrates image resizing.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2

# Load the CSV file
csv_path = 'csv/MAFA_Training_Data_Unstructured.csv'
df = pd.read_csv(csv_path)
image_path = r'C:\Users\alexw\OneDrive\Documents\03_Education\University_Programming\Python\Big_Data\Coursework\Datasets\MAFA\MAFA-Label-Train\train-images'

# Function to resize image and update labels
def resize_image_and_labels(row, target_size, output_dir=None):
    # Load the image
    img_name = row['imgName']
    img = cv2.imread(image_path + '\\' + img_name)
    
    # Resize image
    resized_img = cv2.resize(img, target_size)
    
    # Calculate scaling factors for resizing
    fx = target_size[0] / img.shape[1]
    fy = target_size[1] / img.shape[0]
    
    # Update face and occluder coordinates
    row['face_x'] = np.round(row['face_x'] * fx).astype(int)
    row['face_y'] = np.round(row['face_y'] * fy).astype(int)
    row['face_w'] = np.round(row['face_w'] * fx).astype(int)
    row['face_h'] = np.round(row['face_h'] * fy).astype(int)
    
    row['eye1_x'] = np.round(row['eye1_x'] * fx).astype(int)
    row['eye1_y'] = np.round(row['eye1_y'] * fy).astype(int)
    row['eye2_x'] = np.round(row['eye2_x'] * fx).astype(int)
    row['eye2_y'] = np.round(row['eye2_y'] * fy).astype(int)
    
    row['occluder_x'] = np.round(row['occluder_x'] * fx).astype(int)
    row['occluder_y'] = np.round(row['occluder_y'] * fy).astype(int)
    row['occluder_w'] = np.round(row['occluder_w'] * fx).astype(int)
    row['occluder_h'] = np.round(row['occluder_h'] * fy).astype(int)

    return resized_img, row

# Function to display image with bounding boxes and eye positions
def display_resized_image_with_features(img, row):
    # Display the image
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    
    # Draw face bounding box
    plt.gca().add_patch(patches.Rectangle((row['face_x'], row['face_y']), row['face_w'], row['face_h'], linewidth=2, edgecolor='r', facecolor='none'))
    
    # Draw occluder bounding box
    plt.gca().add_patch(patches.Rectangle((row['face_x'] + row['occluder_x'], row['face_y'] + row['occluder_y']), row['occluder_w'], row['occluder_h'], linewidth=2, edgecolor='b', facecolor='none'))
    
    # Draw glasses bounding box
    plt.gca().add_patch(patches.Rectangle((row['face_x'] + row['glasses_x'], row['face_y'] + row['glasses_y']), row['glasses_w'], row['glasses_h'], linewidth=2, edgecolor='g', facecolor='none'))
    
    # Draw crosses for the positions of the eyes
    plt.plot(row['eye1_x'], row['eye1_y'], 'g+', markersize=15)  # Green cross for eye 1
    plt.plot(row['eye2_x'], row['eye2_y'], 'g+', markersize=15)  # Green cross for eye 2
    
    # Determine whether the person is wearing a mask or not
    occ_type = row['occluder_type']
    plt.title('Simple Occluder' if occ_type == 1 else 'Complex Occluder' if occ_type == 2 else 'Human Body' if occ_type == 3 else 'Unknown Occluder Type')

# Display random images with bounding boxes and eye positions
def resize_random_faces():
    plt.figure(figsize=(15, 10))
    for i in range(5):
        random_idx = np.random.randint(len(df))  # Randomly select an index
        row = df.iloc[random_idx].copy()  # Get a copy of the row
        resized_img, row = resize_image_and_labels(row, (224, 224))
        plt.subplot(5, 5, i+1)
        display_resized_image_with_features(resized_img, row)
        plt.axis('off')

    plt.show()
    
def resize_selected_faces(rows):
    plt.figure(figsize=(15, 10))
    for i, idx in enumerate(rows, start=1):
        row = df.iloc[idx].copy()
        resized_img, row = resize_image_and_labels(row, (224, 224))
        plt.subplot(1, 5, i)
        display_resized_image_with_features(resized_img, row)
        plt.axis('off')
        
    plt.show()
    
# random_idx = np.random.randint(len(df))  # Randomly select an index
# rows =- df.iloc[random_idx].copy()
rows = [0, 1, 2, 3, 4]
resize_selected_faces(rows)
# resize_selected_faces(rows)
# This currently moves the location of the bbxs each times it runs and i'm not entirely sure why. Also the face of 4 is being shifted up for some reason.


In [ ]:
# Testing the structure of the csv for both training and testing data, where multiple faces were stored as multiple arrays within an array. THIS WORKS BUT NOT SOMETHING I THINK I WANT...
# But i think the issue is that we can't pull the data out as easily as when we store them each on seperate rows...It becomes more convoluted.

import scipy.io
import pandas as pd

# Load the .mat files
test_data = scipy.io.loadmat(r'C:\Users\alexw\OneDrive\Documents\03_Education\University_Programming\Python\Big_Data\Coursework\Datasets\MAFA\MAFA-Label-Test\LabelTestAll.mat')
train_data = scipy.io.loadmat(r'C:\Users\alexw\OneDrive\Documents\03_Education\University_Programming\Python\Big_Data\Coursework\Datasets\MAFA\MAFA-Label-Train\LabelTrainAll.mat')

# Convert the test data to a DataFrame
test_label_data = test_data['LabelTest'][0]
test_rows = []
for img_name, label_array in test_label_data:
    img_name = img_name[0]
    face_data = label_array.tolist()  # Convert label array to list
    test_rows.append([img_name, face_data])

# Create a DataFrame for test data
test_df = pd.DataFrame(test_rows, columns=['imgName', 'faces'])

# Save the test DataFrame to a CSV file
test_df.to_csv('csv/MAFA_test_data.csv', index=False)
print("Test CSV file has been successfully generated.")

# Convert the training data to a DataFrame
train_label_data = train_data['label_train'][0]
train_rows = []
for item in train_label_data:
    org_img_name = item[0]
    img_name = item[1]
    label_array = item[2]
    face_data = label_array.tolist()  # Convert label array to list
    train_rows.append([img_name[0], face_data])

# Create a DataFrame for training data
train_df = pd.DataFrame(train_rows, columns=['imgName', 'faces'])

# Save the training DataFrame to a CSV file
train_df.to_csv('csv/MAFA_training_data.csv', index=False)
print("Training CSV file has been successfully generated.")


In [ ]:
# Adds bbx to the one above. Rotated bbx doesn't work - misplaced.

def augment_and_plot(row):
    # Load the image
    img_name = row['imgName']
    img = cv2.imread(image_path + '\\' + img_name)
    
    # Convert image to RGB for displaying with matplotlib
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Plot the original image with bounding box
    plt.figure(figsize=(15, 6))
    plt.subplot(1, 5, 1)
    plt.imshow(img_rgb)
    plt.title('Original')
    plt.axis('off')
    draw_bounding_box(row, img.shape[1], img.shape[0])
    
    # Augmentation 1: Rotation
    rotation_angle = 30
    rotation_matrix = cv2.getRotationMatrix2D((img.shape[1] / 2, img.shape[0] / 2), rotation_angle, 1)
    rotated_img = cv2.warpAffine(img, rotation_matrix, (img.shape[1], img.shape[0]))
    plt.subplot(1, 5, 2)
    plt.imshow(cv2.cvtColor(rotated_img, cv2.COLOR_BGR2RGB))
    plt.title('Rotated')
    plt.axis('off')
    draw_bounding_box(row, img.shape[1], img.shape[0], rotation_angle)
    
    # Augmentation 2: Blur
    blurred_img = cv2.GaussianBlur(img, (15, 15), 0)
    plt.subplot(1, 5, 3)
    plt.imshow(cv2.cvtColor(blurred_img, cv2.COLOR_BGR2RGB))
    plt.title('Blurred')
    plt.axis('off')
    draw_bounding_box(row, img.shape[1], img.shape[0])
    
    # Augmentation 3: Flipping
    flipped_img = cv2.flip(img, 1)  # 1 for horizontal flip
    plt.subplot(1, 5, 4)
    plt.imshow(cv2.cvtColor(flipped_img, cv2.COLOR_BGR2RGB))
    plt.title('Flipped')
    plt.axis('off')
    draw_bounding_box(row, img.shape[1], img.shape[0], flip=True)
    
    # Augmentation 4: Brightness adjustment
    brightness_adjusted_img = cv2.convertScaleAbs(img, alpha=2.0, beta=50)
    plt.subplot(1, 5, 5)
    plt.imshow(cv2.cvtColor(brightness_adjusted_img, cv2.COLOR_BGR2RGB))
    plt.title('Brightness Adjusted')
    plt.axis('off')
    draw_bounding_box(row, img.shape[1], img.shape[0])
    
    plt.tight_layout()
    plt.show()

# Function to draw bounding boxes
def draw_bounding_box(row, img_width, img_height, rotation_angle=0, flip=False):
    face_x, face_y, face_w, face_h = row['face_x'], row['face_y'], row['face_w'], row['face_h']
    occluder_x, occluder_y, occluder_w, occluder_h = row['occluder_x'], row['occluder_y'], row['occluder_w'], row['occluder_h']
    glasses_x, glasses_y, glasses_w, glasses_h = row['glasses_x'], row['glasses_y'], row['glasses_w'], row['glasses_h']
    
    # Adjust occluder coordinates relative to the face bounding box
    occluder_x += face_x
    occluder_y += face_y
    
    if rotation_angle != 0:
        # Rotate bounding boxes
        face_x, face_y = rotate_point(face_x, face_y, img_width / 2, img_height / 2, rotation_angle)
        occluder_x, occluder_y = rotate_point(occluder_x, occluder_y, img_width / 2, img_height / 2, rotation_angle)
        glasses_x, glasses_y = rotate_point(glasses_x, glasses_y, img_width / 2, img_height / 2, rotation_angle)
    
    if flip:
        # Flip bounding boxes
        face_x = img_width - face_x - face_w
        occluder_x = img_width - occluder_x - occluder_w
        glasses_x = img_width - glasses_x - glasses_w
    
    plt.gca().add_patch(patches.Rectangle((face_x, face_y), face_w, face_h, linewidth=2, edgecolor='r', facecolor='none'))
    plt.gca().add_patch(patches.Rectangle((occluder_x, occluder_y), occluder_w, occluder_h, linewidth=2, edgecolor='b', facecolor='none'))
    plt.gca().add_patch(patches.Rectangle((glasses_x, glasses_y), glasses_w, glasses_h, linewidth=2, edgecolor='g', facecolor='none'))


# Function to rotate a point around an origin
def rotate_point(x, y, cx, cy, angle):
    angle_rad = np.deg2rad(angle)
    x_new = (x - cx) * np.cos(angle_rad) - (y - cy) * np.sin(angle_rad) + cx
    y_new = (x - cx) * np.sin(angle_rad) + (y - cy) * np.cos(angle_rad) + cy
    return x_new, y_new

# Testing the function
# Assume `df` is your DataFrame containing image data
row_index = 0  # Choose a specific row for testing
row = df.iloc[row_index]
augment_and_plot(row)


Deprecated

In [ ]:
# Dimensionality Reduction PCA...Not sure what it does quite yet...

import pandas as pd
import numpy as np
import scipy.io
from PIL import Image
import imagehash
import os
import logging
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

if not os.path.exists('logs'):
    os.makedirs('logs')

# Load the .mat file
train_data = scipy.io.loadmat(r'C:\Users\alexw\OneDrive\Documents\03_Education\University_Programming\Python\Big_Data\Coursework\Datasets\MAFA\MAFA-Label-Train\LabelTrainAll.mat')
image_dir = r'C:\Users\alexw\OneDrive\Documents\03_Education\University_Programming\Python\Big_Data\Coursework\Datasets\MAFA\MAFA-Label-Train\train-images'

# Convert the data to a DataFrame
df = pd.DataFrame(train_data['label_train'][0])

# Preprocess DataFrame to remove square brackets
df['orgImgName'] = df['orgImgName'].apply(lambda x: x[0])
df['imgName'] = df['imgName'].apply(lambda x: x[0])
df['label'] = df['label'].apply(lambda x: ', '.join(map(str, x[0])))

### Missing Value validation

# Identify missing values
missing_values = df.isnull().sum()
print("Missing Values:")
print(missing_values)

# Fill in the missing values with the mean value.
# df.fillna(df.mean(), inplace=True) # Cannot be done with images, thus we just remove the row completely.
df.dropna(inplace=True)

### Validating the values column to ensure correct number of elements

# Validate the number of elements in the 'label' column
valid_label_count = df['label'].apply(lambda x: len(x.split(', ')) == 21)
missing_values = valid_label_count.value_counts()[True]  # Count missing values
print("\nNumber of elements with expected label array size:", missing_values)
# print("Missing values:", missing_values)

### Double checking for duplicate values - Does nothing with them, just a note at the moment.

logging.basicConfig(filename='logs\duplicate_images.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def find_duplicate_images(image_dir):
    """
    Find duplicate images in a directory based on their perceptual hashes and log them.
    """
    # Dictionary to store hashes and corresponding filenames
    hash_dict = {}
    
    # List to store duplicate image filenames
    duplicate_images = []
    
    # Iterate through all files in the directory
    for filename in os.listdir(image_dir):
        # Check if the file is an image
        if filename.endswith((".jpg", ".png")):
            # Get the full file path
            filepath = os.path.join(image_dir, filename)
            
            # Open the image and calculate its hash
            try:
                with Image.open(filepath) as img:
                    hash_val = str(imagehash.average_hash(img))
            except Exception as e:
                logging.error(f"Error processing image {filename}: {e}")
                continue
            
            # Check if the hash already exists in the dictionary
            if hash_val in hash_dict:
                # Add the current filename and the filename already stored in hash_dict
                duplicate_images.append((filename, hash_dict[hash_val]))
                logging.info(f"Duplicate image found: {filename} is a duplicate of {hash_dict[hash_val]}")
            else:
                # Add the hash and filename to the dictionary
                hash_dict[hash_val] = filename
                
    # Log the results
    if duplicate_images:
        logging.info(f"Total {len(duplicate_images)} duplicate images found.")
    else:
        logging.info("No duplicate images found.")

# Call the function to find and print duplicate images
# find_duplicate_images(image_dir) # RE-ENABLE THIS IN ORDER TO FIND DUPLICATE IMAGES

### Structure the data and convert into a csv

# Extract bounding box information
label_columns = ['face_x', 'face_y', 'face_w', 'face_h', 'eye1_x', 'eye1_y', 'eye2_x', 'eye2_y', 'occluder_x', 'occluder_y', 'occluder_w', 'occluder_h', 'occ_type', 'occ_degree', 'gender', 'race', 'orientation', 'glasses_x', 'glasses_y', 'glasses_w', 'glasses_h']
df[label_columns] = df['label'].str.split(', ', expand=True)

# Create separate columns for bounding boxes
df['face_bbx'] = df.apply(lambda row: ', '.join(row[['face_x', 'face_y', 'face_w', 'face_h']].astype(str)), axis=1)
df['left_eye_bbx'] = df.apply(lambda row: ', '.join(row[['eye1_x', 'eye1_y']].astype(str)), axis=1)
df['right_eye_bbx'] = df.apply(lambda row: ', '.join(row[['eye2_x', 'eye2_y']].astype(str)), axis=1)
df['occluder_bbx'] = df.apply(lambda row: ', '.join(row[['occluder_x', 'occluder_y', 'occluder_w', 'occluder_h']].astype(str)), axis=1)
df['occluder_type'] = df['occ_type']
df['occluder_degree'] = df['occ_degree']
# df['face_gender'] = df['gender'] # Removing these columns is a form of dimensionality reduction
# df['face_race'] = df['race']
# df['face_orientation'] = df['orientation']
df['glasses_bbx'] = df.apply(lambda row: ', '.join(row[['glasses_x', 'glasses_y', 'glasses_w', 'glasses_h']].astype(str)), axis=1)

# Drop the original columns
df.drop(columns=label_columns, inplace=True)

# Convert the extracted columns to numeric type
df[['face_bbx', 'left_eye_bbx', 'right_eye_bbx', 'occluder_bbx', 'occluder_type', 'occluder_degree', 'glasses_bbx']] = df[['face_bbx', 'left_eye_bbx', 'right_eye_bbx', 'occluder_bbx', 'occluder_type', 'occluder_degree', 'glasses_bbx']].apply(lambda x: pd.to_numeric(x, errors='coerce'))

# Drop the original 'label' column
df.drop(columns=['label'], inplace=True)

# Exclude non-numeric columns from DataFrame before standardization
non_numeric_columns = ['orgImgName', 'imgName']
numeric_columns = [col for col in df.columns if col not in non_numeric_columns]

# Standardize the data
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df[numeric_columns])

# Impute missing values with the mean
imputer = SimpleImputer(strategy='mean')
df_imputed = imputer.fit_transform(df_scaled)

# Apply PCA
pca = PCA(n_components=0.5)  # Keep 95% of variance
df_pca = pca.fit_transform(df_imputed)

# Convert PCA results back to DataFrame
df_pca = pd.DataFrame(df_pca, columns=[f'PC{i+1}' for i in range(df_pca.shape[1])])

# Save the DataFrame to a CSV file without square brackets
df_pca.to_csv('csv/MAFA_Label_Train_PCA.csv', index=False)

# Display the cleaned DataFrame
print("\nDataFrame after PCA:")
print(df_pca.head())

In [ ]:
# Puts the train data (unstructured) into a csv in order to apply dimensionality reduction on it.

# Load the .mat file
train_data = scipy.io.loadmat(r'C:\Users\alexw\OneDrive\Documents\03_Education\University_Programming\Python\Big_Data\Coursework\Datasets\MAFA\MAFA-Label-Train\LabelTrainAll.mat')
image_dir = r'C:\Users\alexw\OneDrive\Documents\03_Education\University_Programming\Python\Big_Data\Coursework\Datasets\MAFA\MAFA-Label-Train\train-images'

# Convert the data to a DataFrame
df = pd.DataFrame(train_data['label_train'][0])

# Preprocess DataFrame to remove square brackets
df['orgImgName'] = df['orgImgName'].apply(lambda x: x[0])
df['imgName'] = df['imgName'].apply(lambda x: x[0])
df['label'] = df['label'].apply(lambda x: ', '.join(map(str, x[0])))

# Structure the data
label_columns = ['face_x', 'face_y', 'face_w', 'face_h', 'eye1_x', 'eye1_y', 'eye2_x', 'eye2_y', 
                 'occluder_x', 'occluder_y', 'occluder_w', 'occluder_h', 'occluder_type', 'occluder_degree', 
                 'gender', 'race', 'orientation', 'glasses_x', 'glasses_y', 'glasses_w', 'glasses_h']
df[label_columns] = df['label'].str.split(', ', expand=True)

# Convert the bounding box coordinates from strings to floats
for col in label_columns:
    df[col] = df[col].astype(float)

df.drop(columns=['label'], inplace=True)

# Define columns for the CSV
csv_columns = ['orgImgName', 'imgName', 'face_x', 'face_y', 'face_w', 'face_h', 
               'eye1_x', 'eye1_y', 'eye2_x', 'eye2_y', 'occluder_x', 'occluder_y', 'occluder_w', 'occluder_h', 'occluder_type', 'occluder_degree',
               'gender', 'race', 'orientation', 'glasses_x', 'glasses_y', 'glasses_w', 'glasses_h']

# Save the DataFrame to a CSV file
df[csv_columns].to_csv('csv/MAFA_training_data_unstructured.csv', index=False)

print("CSV file has been successfully generated.")

In [ ]:
# LDA - Linear Discriminant Analysis - Dimensionality Reduction 

import pandas as pd
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# Load your dataset
df = pd.read_csv(r'C:\Users\alexw\OneDrive\Documents\03_Education\University_Programming\Python\Big_Data\Coursework\Masked_Face\csv\MAFA_Label_Train.csv')

# Drop irrelevant features
df.drop(columns=['orgImgName', 'imgName', 'glasses_bbx'], inplace=True)

# Filter out occluder_type equal to 3
df = df[df['occluder_type'].isin([1, 2])]

# Convert string features to numeric
df[['face_bbx', 'left_eye_bbx', 'right_eye_bbx', 'occluder_bbx']] = df[['face_bbx', 'left_eye_bbx', 'right_eye_bbx', 'occluder_bbx']].apply(lambda x: x.str.split(',').apply(lambda y: [float(val) for val in y]))

# Expand the columns containing lists into separate columns
df = pd.concat([df.drop(columns=['face_bbx', 'left_eye_bbx', 'right_eye_bbx', 'occluder_bbx']), df['face_bbx'].apply(pd.Series).add_suffix('_face'), df['left_eye_bbx'].apply(pd.Series).add_suffix('_left_eye'), df['right_eye_bbx'].apply(pd.Series).add_suffix('_right_eye'), df['occluder_bbx'].apply(pd.Series).add_suffix('_occluder')], axis=1)

# Separate features and target variable
X = df.drop(columns=['occluder_type'])
y = df['occluder_type']

# Initialize and fit LDA with reduced number of components
n_classes = len(y.unique())
n_features = X.shape[1]
n_components = min(n_features, n_classes - 1)  # Ensure valid number of components
lda = LinearDiscriminantAnalysis(n_components=n_components)
X_lda = lda.fit_transform(X, y)

# Create a DataFrame for the transformed data
df_lda = pd.DataFrame(data=X_lda, columns=[f'LD{i}' for i in range(1, n_components + 1)])

# Concatenate the transformed data with the target variable
df_lda['occluder_type'] = y.reset_index(drop=True)

# Save the transformed data to a new CSV file
df_lda.to_csv('csv\lda_transformed_data.csv', index=False)

In [ ]:
# Plotting the LDA results

import pandas as pd
import matplotlib.pyplot as plt

# Load the data
df_lda = pd.read_csv('csv\lda_transformed_data.csv')

# Separate data points for each class
class1 = df_lda[df_lda['occluder_type'] == 1.0]
class2 = df_lda[df_lda['occluder_type'] == 2.0]

# Plot the data points for each class
plt.scatter(class1['LD1'], [1] * len(class1), label='occluder_type 1.0', color='blue')
plt.scatter(class2['LD1'], [2] * len(class2), label='occluder_type 2.0', color='red')

# Add labels and title
plt.xlabel('LD1')
plt.yticks([1, 2], ['occluder_type 1.0', 'occluder_type 2.0'])
plt.title('Scatter Plot of LD1 vs occluder_type')
plt.legend()

# Show plot
plt.show()


In [ ]:
# Attempt 2 to at logistic regression model: https://www.justintodata.com/logistic-regression-example-in-python/

import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, roc_auc_score, recall_score, precision_score, average_precision_score, f1_score, classification_report, accuracy_score

from scipy.io import loadmat

train_data_path = r'C:\Users\alexw\OneDrive\Documents\03_Education\University\University_Programming\Python\Big_Data\Coursework\Datasets\MAFA\MAFA-Label-Train\LabelTrainAll.mat'

### Explore and clean the data ###

# Read the lines from the text file
with open(train_data_path, 'r') as file:
    lines = file.readlines()

# Initialize lists to store data
image_paths = []
num_faces_list = []
face_info_list = []

# Process lines to extract information
for line in lines:
    line = line.strip()

    # Skip empty lines
    if not line:
        continue

    # Check if it represents an image path
    if line.endswith('.jpg'):
        image_paths.append(line)
        # Initialize num_faces and face_info
        num_faces = None
        face_info = []
    else:
        # If it's not an image path, it's face information
        if num_faces is None:
            # Extract the number of faces
            num_faces = int(line)
        else:
            # Split face information
            face_info.extend([int(x) for x in line.split()])

        # Check if we have collected enough face information
        if len(face_info) == num_faces * 10:
            num_faces_list.append(num_faces)
            face_info_list.append(face_info)

# Create a DataFrame
df = pd.DataFrame({
    'Image_Path': image_paths,
    'Num_Faces': num_faces_list,
    'Face_Info': face_info_list
})

# Prints the column names of the DataFrame
# df.columns # OUTPUT: Index(['Image_Path', 'Num_Faces', 'Face_Info'], dtype='object')

# Display the DataFrame
# print(df.head())

# This code can be used to find out how many pictures have 1 face, how many have 2 faces...singles out one of the data types
# df = df.rename(columns={'Num_Faces': 'target'})
# df['target'].value_counts(dropna=False)

# Lists a summary of the dataframe
# df.info()

# If df.info() returns columns with wildly different values, the below code will remove the ones with many missing values
# df = df.drop(['slope', 'ca', 'thal'], axis=1) # slope, ca, and thal were values from the example
# df = df.dropna().copy()

# Nice summary of the dataframe
df

In [ ]:
import scipy.io
import os
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Load the .mat file
mat_data = scipy.io.loadmat(r'C:\Users\alexw\OneDrive\Documents\03_Education\University\University_Programming\Python\Big_Data\Coursework\Datasets\MAFA\MAFA-Label-Train\LabelTrainAll.mat')

# Extract the labels
label_data = mat_data['label_train'][0]

# Assuming you know the image file names, you can list them from the train_images directory
image_files_path = r'C:\Users\alexw\OneDrive\Documents\03_Education\University\University_Programming\Python\Big_Data\Coursework\Datasets\MAFA\train-images\images'
image_files = os.listdir(image_files_path)

# Ensure num_samples is not greater than the number of available images and labels
num_samples = min(5, len(image_files), len(label_data))  # Number of sample images to display

# Randomly select indices for sampling
random_indices = np.random.choice(len(image_files), num_samples, replace=False)

plt.figure(figsize=(15, 10))
for i, random_idx in enumerate(random_indices, 1):
    image_file = image_files[random_idx]
    image_path = os.path.join(image_files_path, image_file)
    image = plt.imread(image_path)
    plt.subplot(1, num_samples, i)
    plt.imshow(image)
    
    # Extract labels for the current image
    org_img_name = label_data[random_idx][0][0]
    img_name = label_data[random_idx][1][0]
    label = label_data[random_idx][2][0]
    
    # Get the face bounding box
    face_bbox = label[:4]
    x, y, w, h = face_bbox
    
    # Draw face bounding box
    plt.gca().add_patch(patches.Rectangle((x, y), w, h, linewidth=2, edgecolor='r', facecolor='none'))
    
    # Get the bounding box of the occluder
    occluder_bbox = label[8:12]
    if any(occluder_bbox != -1):
        ox, oy, ow, oh = occluder_bbox
        # Draw occluder bounding box
        plt.gca().add_patch(patches.Rectangle((x + ox, y + oy), ow, oh, linewidth=2, edgecolor='b', facecolor='none'))
    
    # Get the bounding box of the glasses
    glasses_bbox = label[-4:]
    if any(glasses_bbox != -1):
        gx, gy, gw, gh = glasses_bbox
        # Draw glasses bounding box
        plt.gca().add_patch(patches.Rectangle((x + gx, y + gy), gw, gh, linewidth=2, edgecolor='g', facecolor='none'))
    
    # Determine whether the person is wearing a mask or not
    occ_type = label[12]
    plt.title('Mask' if occ_type == 1 else 'No Mask')
    plt.axis('off')

plt.show()
